# Libraries

In [1]:
import pandas as pd
import numpy as np
import re
import geonamescache 
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer
import pickle

import matplotlib.pyplot as plt
import seaborn as sns


# Load Data

## Load SQL

In [2]:
my_studies = pd.read_pickle(r".\df_sql\my_studies.pkl")
my_conditions = pd.read_pickle(r".\df_sql\my_conditions.pkl")  
my_covid = pd.read_pickle(r".\df_sql\my_covid.pkl")  
my_placebo = pd.read_pickle(r".\df_sql\my_placebo.pkl")  
my_interventions = pd.read_pickle(r".\df_sql\my_interventions.pkl")  
my_interventions_types = pd.read_pickle(r".\df_sql\my_interventions_types.pkl")  
my_soc = pd.read_pickle(r".\df_sql\my_soc.pkl")  
my_intervention_methods2 = pd.read_pickle(r".\df_sql\my_intervention_methods2.pkl")  
my_intervention_methods = pd.read_pickle(r".\df_sql\my_intervention_methods.pkl")  
my_adverse = pd.read_pickle(r".\df_sql\my_adverse.pkl")  
my_adverse_system = pd.read_pickle(r".\df_sql\my_adverse_system.pkl")  
my_designs = pd.read_pickle(r".\df_sql\my_designs.pkl")  
my_eligibilities = pd.read_pickle(r".\df_sql\my_eligibilities.pkl")  
my_locations = pd.read_pickle(r".\df_sql\my_locations.pkl")  


## Load Data

In [3]:
df = pd.read_csv(r".\thesis_data\ctg-studies.csv")
pd.set_option('display.max.columns', 35)  # Number of Columns : 35 > 30
df = df.rename(columns = {'NCT Number': 'nct_id'})
df = df.merge(my_studies[['nct_id', 'official_title']], on = 'nct_id', how = 'left')

display(df.shape)
df.head()

(79217, 31)

,nct_id,Study Title,Study URL,Acronym,Study Status,Brief Summary,Study Results,Conditions,Interventions,Primary Outcome Measures,Secondary Outcome Measures,Other Outcome Measures,Sponsor,Collaborators,Sex,Age,Phases,Enrollment,Funder Type,Study Type,Study Design,Other IDs,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents,official_title
0,NCT02064621,CKD-330 Drug-Drug Interaction Study (Candesartan),https://clinicaltrials.gov/study/NCT02064621,CKD-330(C),COMPLETED,The purpose of this study is to evaluate a pha...,NO,Hypertension,DRUG: Candesartan 32mg|DRUG: Candesartan 32mg/...,"AUCτ,ss of Candesartan, 1D 0h, 8D 0h, 9D 0h, 1...","Tmax,ss of Candesartan, 1D 0h, 8D 0h, 9D 0h, 1...",NaN,Chong Kun Dang Pharmaceutical,NaN,MALE,ADULT,PHASE1,24.0,INDUSTRY,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: CRO...,144HPS13013,2014-06,2014-08,2014-08,2014-02-17,NaN,2014-09-03,"Asan Medical Center, Seoul, Songpa-gu, South K...",NaN,"A Randomized, Open-label, Multiple Doses, Cros..."
1,NCT01756521,A Study Study to Investigate the Effect on QT/...,https://clinicaltrials.gov/study/NCT01756521,NaN,COMPLETED,The purpose of this study is to evaluate effec...,NO,the Effect of Moxifloxacin on QT/QTc Interval,DRUG: Moxifloxacin,"QTc (QTcF, QTcB, QTcI), QTc (QTcF, QTcB, QTcI)...",NaN,NaN,Seoul National University Hospital,Korea National Enterprise for Clinical Trials|...,MALE,ADULT,PHASE1,39.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: CRO...,MOXI036,2013-01,2013-08,2013-10,2012-12-27,NaN,2016-04-26,"Seoul National University Hospital, Seoul, Sou...",NaN,"A Multi-center, Randomized, Open-label, Placeb..."
2,NCT03318614,Bifidobacterium Infantis M-63 Improves Mental ...,https://clinicaltrials.gov/study/NCT03318614,NaN,COMPLETED,A 3-month study was conducted in flood victims...,NO,Irritable Bowel Syndrome|Abdominal Pain|Small ...,DIETARY_SUPPLEMENT: Probiotics M-63|OTHER: Con...,"IBS Symptom Severity Scale (IBS-SSS) scores, I...","36-Item Short Form Survey (SF-36) scores, SF-3...",NaN,Universiti Sains Malaysia,"Morinaga Milk Industry Co., LTD",ALL,"ADULT, OLDER_ADULT",PHASE2|PHASE3,53.0,OTHER,INTERVENTIONAL,Allocation: NON_RANDOMIZED|Intervention Model:...,USM/JEPeM/15040133,2015-09,2015-12,2015-12,2017-10-24,NaN,2017-10-24,NaN,NaN,Bifidobacterium Infantis M-63 Improves Mental ...
3,NCT03761914,Galinpepimut-S in Combination With Pembrolizum...,https://clinicaltrials.gov/study/NCT03761914,NaN,COMPLETED,"To evaluate the safety, tolerability, and anti...",YES,Acute Myelogenous Leukemia|Ovarian Cancer|Colo...,BIOLOGICAL: galinpepimut-S|BIOLOGICAL: pembrol...,Overall Response Rate (ORR) - Best Overall Res...,NaN,Progression-free Survival (PFS) - Kaplan Meier...,Sellas Life Sciences Group,Merck Sharp & Dohme LLC|LumaBridge,ALL,"ADULT, OLDER_ADULT",PHASE1|PHASE2,26.0,INDUSTRY,INTERVENTIONAL,Allocation: NON_RANDOMIZED|Intervention Model:...,SLS17-201/MK3475-770,2019-09-30,2022-07-05,2022-10-27,2018-12-03,2024-11-19,2024-11-19,"St. Joseph Heritage Healthcare, Santa Rosa, Ca...","Study Protocol and Statistical Analysis Plan, ...",A Phase 1/2 Study of Galinpepimut-S in Combina...
4,NCT05340504,Evaluating the Hypothesized Mechanism of Actio...,https://clinicaltrials.gov/study/NCT05340504,NaN,COMPLETED,This research study evaluates the effects of a...,YES,Bipolar Disorder|Bipolar I Disorder|Bipolar II...,DRUG: N-Acetyl cysteine|DRUG: Placebo,Change in dACC GSH Levels Through Proton Magne...,NaN,NaN,Medical University of South Carolina,Milken Institute,ALL,ADULT,PHASE2,12.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: CRO...,00117736,2022-05-13,2023-06-21,2023-06-21,2022-04-22,2024-06-26,2024-06-26,"Medical University Of South Carolina, Charlest...","Study Protocol and Statistical Analysis Plan, ...",Experimentally Evaluating the Hypothesized Mec...


##  Drop Rows
Exclude: 
- clinical trials missing from db but exist in csv.



In [4]:
display(df.shape)

# Drop studies that are not in my_studies table of db and via versa.
display(df.loc[~df["nct_id"].isin(my_studies["nct_id"])].shape[0])
df = df.drop(df.loc[~df["nct_id"].isin(my_studies["nct_id"])].index)  # Those not in PHASE1|2, 2|3 will also going to be dropped

display(df['nct_id'].shape[0] - df["nct_id"].isin(my_studies["nct_id"]).shape[0])
df = df[df["nct_id"].isin(my_studies["nct_id"])]

display(df.shape)


(79217, 31)

1

0

(79216, 31)

### Duplicated Studies

- nct_id always unique --> given from Site page

ClinicalTrials.gov Documentation:
•	NCT number
A unique identification code given to each clinical study record registered on ClinicalTrials.gov. The format is "NCT" followed by an 8-digit number (for example, NCT00000419). Also called the ClinicalTrials.gov identifier.

ClinicalTrials.gov Documentation:
•	Other study IDs
Identifiers or ID numbers other than the NCT number that are assigned to a clinical study by the study's sponsor, funders, or others. These numbers may include unique identifiers from other trial registries and National Institutes of Health grant numbers.

In [5]:
display(df.duplicated().value_counts())  

display(df.describe(include = ['object'])[[
    "nct_id", 'Study Status', 'Phases', "Study Title", "official_title", 'Start Date', 'Completion Date'
    ]])
display(df[df.duplicated(subset = [
    'Study Status', 'Phases', "Study Title", 'official_title', 'Brief Summary', 'Start Date', 'Completion Date'
    ], keep = False)][[
        'Study Status', 'Phases', "Study Title", 'official_title', 'Brief Summary', 'Start Date', 'Completion Date'
        ]].sort_values(by = "Study Title"))  

df = df.drop_duplicates(subset = [
    'Study Status', 'Phases', "Study Title", 'official_title', 'Brief Summary', 'Start Date', 'Completion Date'
    ], keep = 'first').reset_index(drop = True)

display(df.shape)

False    79216
Name: count, dtype: int64

,nct_id,Study Status,Phases,Study Title,official_title,Start Date,Completion Date
count,79216,79216,79216,79216,78268,79216,79216
unique,79216,3,6,78998,77819,4738,4456
top,NCT02064621,COMPLETED,PHASE1,Multi-Center Study of New Medications to Treat...,ACTIV-6: COVID-19 Outpatient Randomized Trial ...,2012-01,2014-12
freq,1,63543,22233,3,8,653,659


,Study Status,Phases,Study Title,official_title,Brief Summary,Start Date,Completion Date
2648,TERMINATED,PHASE3,A Phase 3 Safety and Efficacy Study of Fovista...,"A Phase 3 Randomized, Double-masked, Controlle...",The objectives of this study are to evaluate t...,2013-08,2016-12
71430,TERMINATED,PHASE3,A Phase 3 Safety and Efficacy Study of Fovista...,"A Phase 3 Randomized, Double-masked, Controlle...",The objectives of this study are to evaluate t...,2013-08,2016-12
3538,COMPLETED,PHASE3,A Study of the Efficacy and Safety of DFD-06 C...,"A Randomized, Double-Blind, Vehicle-Controlled...",This study will compare the efficacy and safet...,2015-12,2016-07
49258,COMPLETED,PHASE3,A Study of the Efficacy and Safety of DFD-06 C...,"A Randomized, Double-Blind, Vehicle-Controlled...",This study will compare the efficacy and safet...,2015-12,2016-07
53348,COMPLETED,PHASE3,Ceftazidime-Avibactam Compared With Doripenem ...,"A Phase III, Randomized, Multicenter, Double-B...",The purpose of this study is to evaluate the e...,2012-10,2014-08
23595,COMPLETED,PHASE3,Ceftazidime-Avibactam Compared With Doripenem ...,"A Phase III, Randomized, Multicenter, Double-B...",The purpose of this study is to evaluate the e...,2012-10,2014-08
44741,COMPLETED,PHASE1|PHASE2,Effect of Bronchoscopy on the Outcome of Patie...,Effect of Bronchoscopy on the Outcome of Patie...,to evaluate the effect of use of bronchoscopy ...,2020-07-02,2021-12-01
75572,COMPLETED,PHASE1|PHASE2,Effect of Bronchoscopy on the Outcome of Patie...,Effect of Bronchoscopy on the Outcome of Patie...,to evaluate the effect of use of bronchoscopy ...,2020-07-02,2021-12-01
2905,COMPLETED,PHASE1|PHASE2,Effect of Bronchoscopy on the Outcome of Patie...,Effect of Bronchoscopy on the Outcome of Patie...,to evaluate the effect of use of bronchoscopy ...,2020-07-02,2021-12-01
73332,COMPLETED,PHASE3,Efficacy Study of Nebulized TD-4208 for Chroni...,"A Phase 3, 12-week, Randomized, Double-blind P...",The purpose of this study is to measure the ef...,2015-09,2016-09


(79204, 31)

# Columns Format Dtypes

## Phases_List
- 5 phases and NA: 
1. Early Phase 1 (Phase 0) --> Not used. Out of this studies purpose
2. Phase 1, 
3. Phase 2, 
4. Phase 3,  
5. Phase 4. 
6. Not Applicable --> without FDA-defined phases. Regards devices, behavioral trials etc. . --> Not used. Out of this studies purpose



In [6]:
df = df.rename(columns = {'Phases':'Phases_List'})
display(df["Phases_List"].value_counts(dropna=False))  
display(df.shape)

df["Phases_List"] = df["Phases_List"].str.split( "|" ) 
# No apply sort(list(x)) needed here - No duplicated elements with diff list-elem order

display(df['Phases_List'].value_counts(dropna = False))
display(df.shape)

Phases_List
PHASE1           22233
PHASE2           21415
PHASE3           14090
PHASE4           13352
PHASE1|PHASE2     5563
PHASE2|PHASE3     2551
Name: count, dtype: int64

(79204, 31)

Phases_List
[PHASE1]            22233
[PHASE2]            21415
[PHASE3]            14090
[PHASE4]            13352
[PHASE1, PHASE2]     5563
[PHASE2, PHASE3]     2551
Name: count, dtype: int64

(79204, 31)

## Study_Status_Bin
Information Source: https://clinicaltrials.gov/policy/protocol-definitions#study-status


- Withdrawn Studies = Permenantely Stopped as Noted in Data Source. Withdrawn is Replaced as Terminated. 

In [7]:
df = df.rename(columns={"Study Status": "Study_Status_Bin"})
df['Study_Status_Bin'] = df['Study_Status_Bin'].replace({"WITHDRAWN": "TERMINATED"})
df["Study_Status_Bin"] = df["Study_Status_Bin"].astype("category")
display(df['Study_Status_Bin'].value_counts(dropna=False)) 
display(df.shape)

Study_Status_Bin
COMPLETED     63535
TERMINATED    15669
Name: count, dtype: int64

(79204, 31)

## Age_List
- Directly from csv - Structured format
- No NaNs

In [8]:
display(df["Age"].value_counts(dropna=False))  
 
df["Age"] = df["Age"].str.replace(', ', ',').str.split(",")   # No apply sort(list(x)) needed here - No duplicated elements with diff list-elem order

# display(Counter([elem for row_list in df["Age"].dropna() for elem in row_list]))  
# display(set(elem for row_list in df["Age"].dropna() for elem in row_list)) # Unique values in list-type element column

df = df.rename(columns = {"Age": "Age_List"})

display(df["Age_List"].value_counts(dropna = False))  


Age
ADULT, OLDER_ADULT           53235
ADULT                        15101
CHILD                         4038
CHILD, ADULT, OLDER_ADULT     3860
CHILD, ADULT                  2349
OLDER_ADULT                    621
Name: count, dtype: int64

Age_List
[ADULT, OLDER_ADULT]           53235
[ADULT]                        15101
[CHILD]                         4038
[CHILD, ADULT, OLDER_ADULT]     3860
[CHILD, ADULT]                  2349
[OLDER_ADULT]                    621
Name: count, dtype: int64

## Sex_List

In [9]:
df = df.rename(columns = {"Sex": "Sex_List"})

display(df["Sex_List"].value_counts(dropna=False))  

# Fillna from my_eligibilities df
df = df.merge(my_eligibilities[["nct_id", "gender"]], on = "nct_id", how = "left")
df['Sex_List'] = df['Sex_List'].fillna(df['gender'])

# Drop db column
df = df.drop(columns = ['gender'])

df["Sex_List"].isnull().sum()  # Just 1 row/point filled


Sex_List
ALL       68232
FEMALE     5865
MALE       5077
NaN          30
Name: count, dtype: int64

30

### NaNs --> Drop
- Fillna from Brief Summary
- Remaining nulls --> Dropped as they were too little comparing to dataset

In [10]:
# Fillna from Biref Summary
# male + female
df.loc[ df["Sex_List"].isnull() & df["Brief Summary"].str.contains(r"(?i)\bmale\b|\bmen\b", na=False) & \
       df["Brief Summary"].str.contains(r"(?i)\bfemale\b|\bwomen\b", na=False), "Sex_List"] = "ALL"
# only female
df.loc[ df["Sex_List"].isnull() & df["Brief Summary"].str.contains(r"(?i)\bfemale\b|\bwomen\b", na=False) & \
    ~ df["Brief Summary"].str.contains(r"(?i)\bmale\b|\bmen\b", na=False), "Sex_List"] = "FEMALE"
# only male
df.loc[ df["Sex_List"].isnull() & df["Brief Summary"].str.contains(r"(?i)\bmale\b|\bmen\b", na=False) & \
       ~ df["Brief Summary"].str.contains(r"(?i)\bfemale\b|\bwomen\b", na=False), "Sex_List"] = "MALE"

display(df["Sex_List"].isnull().sum()) # too little rows were filled from Brief Summary
# NaNs --> Drop
df = df.dropna(subset=['Sex_List'])
display(df["Sex_List"].isnull().sum())



27

0

## Funder_Industry_Bin
- Directly from csv - Structured format
- No NaNs

PRS Source: https://register.clinicaltrials.gov/prs/html/account-organizations.html#page-top
Companies Source : https://companiesmarketcap.com/pharmaceuticals/largest-pharmaceutical-companies-by-market-cap/
https://clinicaltrials.gov/study-basics/glossary

ClinicalTrials.gov Documentation:
1. U.S. National Institutes of Health
2. Other U.S. Federal agencies (e.g., FDA, Centers for Disease Control and Prevention, or U.S. Department of Veterans Affairs)
3. Industry (for example: pharmaceutical and device companies)
4. All others (including individuals, universities, and community-based organizations)

- Almost all Industry rows = Pharmaceutical

In [11]:
df = df.rename(columns = {"Funder Type": "Funder_Industry_Bin"})
display(df["Funder_Industry_Bin"].value_counts(dropna = False))  

# Merging Categories into more general 
# Some categories have too little samples (e.g., Thesis Vizual File)
funder_map = {
    'OTHER': 'FUNDER_OTHER',
    'NETWORK': 'FUNDER_OTHER', 
    'INDIV': 'FUNDER_OTHER',
    'UNKNOWN': 'FUNDER_OTHER',
    'AMBIG': 'FUNDER_OTHER',
    'INDUSTRY': 'INDUSTRY',
    'OTHER_GOV': 'FUNDER_OTHER',  # at great risk of sparsity even if combined with FED and NIH
    'FED': 'FUNDER_OTHER',
    'NIH': 'FUNDER_OTHER'
}
df['Funder_Industry_Bin'] = df['Funder_Industry_Bin'].map(funder_map)

display(df["Funder_Industry_Bin"].value_counts(dropna=False))  
display(df.shape)

Funder_Industry_Bin
INDUSTRY     39525
OTHER        36193
NIH           1289
OTHER_GOV     1089
NETWORK        555
FED            416
INDIV          103
UNKNOWN          7
Name: count, dtype: int64

Funder_Industry_Bin
FUNDER_OTHER    39652
INDUSTRY        39525
Name: count, dtype: int64

(79177, 31)

## Datetime
- Source Date input : Filter dates have the format of mm/dd/yyyy. 
- Dates Displayed in Outputs (after filtering choices menu) have the Format of yyyy-mm-dd (or at least yyyy-mm).
- All Datetimes converted to same format : Year-Month is Chosen as Many Dates Include only until Month.


In [12]:
# Dates are in yyyy-mm-dd (len=10) and yyyy-mm (len=7) format
display(df[["Start Date","Completion Date"]].map(len, na_action = 'ignore').value_counts()) 
# display(df[["Start Date","Completion Date"]].map(len, na_action = 'ignore').nunique()) 

df[["Start Date","Completion Date"]] = df[["Start Date","Completion Date"]]\
    .apply(pd.to_datetime, format='%Y-%m', exact=False, errors='coerce')

df[["Start Date","Completion Date"]] = df[["Start Date","Completion Date"]]\
    .apply(lambda col: col.dt.to_period('M'))

display(df[["Start Date","Completion Date"]].head(2))

Start Date  Completion Date
10          10                 47571
7           7                  27042
            10                  3985
10          7                    579
Name: count, dtype: int64

,Start Date,Completion Date
0,2014-06,2014-08
1,2013-01,2013-10


### Datetime_Year

In [13]:
df[["Start_Date_Year_Counts", "Completion_Date_Year_Counts"]] = df[["Start Date", "Completion Date"]]\
    .apply(lambda x: x.dt.year)

df["Start_Date_Year_Counts"] = pd.to_numeric(df["Start_Date_Year_Counts"], errors="coerce").astype("Int64")
df["Completion_Date_Year_Counts"] = pd.to_numeric(df["Completion_Date_Year_Counts"], errors="coerce").astype("Int64")

display(df[["Start_Date_Year_Counts","Completion_Date_Year_Counts"]])


,Start_Date_Year_Counts,Completion_Date_Year_Counts
0,2014,2014
1,2013,2013
2,2015,2015
3,2019,2022
4,2022,2023
...,...,...
79199,2014,2015
79200,2011,2017
79201,2019,2019
79202,2016,2016


### Completion_Gap

In [14]:
df["Completion_Gap_Counts"] = df["Completion Date"] - df["Start Date"]

df[["Completion_Gap_Counts"]] = df[["Completion_Gap_Counts"]]\
    .apply(lambda col: col.astype('str').str.strip('< *MonthEnds>')\
           .replace("NaT", np.NaN).replace("", '0').astype('float'))

df[["Completion_Gap_Counts",]] = df[["Completion_Gap_Counts"]].convert_dtypes(convert_integer = True)

display(df[["Completion_Gap_Counts"]].head(2))


,Completion_Gap_Counts
0,2
1,9


### Completion_Gap_Log

In [15]:
df['Completion_Gap_Log'] = np.log1p(df["Completion_Gap_Counts"]) 

## Intervention_Type_List

Source: https://clinicaltrials.gov/policy/protocol-definitions#arms-groups-interventions

ClinicalTrials.gov Documentation:
Intervention Type:
For each intervention studied in the clinical study, the general type of intervention.

- Drug: Including placebo
- Device: Including sham
- Biological/Vaccine
- Procedure/Surgery
- Radiation
- Behavioral: For example, psychotherapy, lifestyle counseling
- Genetic: Including gene transfer, stem cell and recombinant DNA
- Dietary Supplement: For example, vitamins, minerals
- Combination Product: Combining a drug and device, a biological product and device; a drug and biological product; or a drug,   biological product, and device
- Diagnostic Test: For example, imaging, in vitro
- Other

In [16]:
df = df.merge(my_interventions_types[["nct_id", "intervention_type"]], on = "nct_id", how = "left")
df = df.rename({"intervention_type" : "Intervention_Type_List"}, axis = 1)

df['Intervention_Type_List'] = df['Intervention_Type_List'].apply(lambda x: ['INTERV_OTHER'] if 'OTHER' in x else x)

display(df["Intervention_Type_List"].value_counts(dropna=False))
display(Counter([elem for row_list in df["Intervention_Type_List"].dropna() for elem in row_list]))  
display(df["Intervention_Type_List"].isnull().sum())  # No Null Values

Intervention_Type_List
[DRUG]                                               54796
[INTERV_OTHER]                                        7607
[BIOLOGICAL]                                          5747
[BIOLOGICAL, DRUG]                                    1616
[DEVICE]                                              1559
                                                     ...  
[DIAGNOSTIC_TEST, DRUG, GENETIC]                         1
[BEHAVIORAL, DRUG, GENETIC]                              1
[BIOLOGICAL, COMBINATION_PRODUCT, DEVICE]                1
[DEVICE, PROCEDURE, RADIATION]                           1
[COMBINATION_PRODUCT, DIAGNOSTIC_TEST, PROCEDURE]        1
Name: count, Length: 110, dtype: int64

Counter({'DRUG': 60420,
         'BIOLOGICAL': 7852,
         'INTERV_OTHER': 7607,
         'DEVICE': 2900,
         'PROCEDURE': 2558,
         'BEHAVIORAL': 1312,
         'DIETARY_SUPPLEMENT': 1188,
         'RADIATION': 1115,
         'COMBINATION_PRODUCT': 637,
         'DIAGNOSTIC_TEST': 210,
         'GENETIC': 154})

0

### Merge

In [17]:
cont_map = { 
    'COMBINATION_PRODUCT' : 'INTERV_UNSPES',
    'DIAGNOSTIC_TEST' : 'INTERV_UNSPES',
    'GENETIC' : 'INTERV_UNSPES'   
}

df['Intervention_Type_List'] = df['Intervention_Type_List'].apply(lambda row: [cont_map.get(val, val) for val in row])
display(Counter([elem for row_list in df["Intervention_Type_List"] for elem in row_list]))

Counter({'DRUG': 60420,
         'BIOLOGICAL': 7852,
         'INTERV_OTHER': 7607,
         'DEVICE': 2900,
         'PROCEDURE': 2558,
         'BEHAVIORAL': 1312,
         'DIETARY_SUPPLEMENT': 1188,
         'RADIATION': 1115,
         'INTERV_UNSPES': 1001})

## Intervention_Type_Counts

In [18]:
df["Intervention_Type_Counts"] = df["Intervention_Type_List"].apply(lambda x: len(x)) # no null in Intervention_Type_List
display(df["Intervention_Type_Counts"].value_counts(dropna=False))  
display(df["Intervention_Type_Counts"].isnull().sum())
display(df.shape)

Intervention_Type_Counts
1    72962
2     5695
3      480
4       39
5        1
Name: count, dtype: int64

0

(79177, 37)

## Intervention_Route_List

In [19]:
with open(r".\thesis_data\str_oral_injection_surgical_topical.pkl", "rb") as f:
    data = pickle.load(f)

oral1 = data["oral1"]
oral2 = data["oral2"]
injection1 = data["injection1"]
injection2 = data["injection2"]
topical1 = data["topical1"]
surgical1 = data["surgical1"]

### NaNs --> Fillna text -> Other

In [20]:
## Interventions column
# Injection
df.loc[df['Interventions'].str.contains(injection1, case = False, na = False), 'Injection'] = "Injection"
df.loc[df['Interventions'].str.lower().str.lower().str.contains(injection2, case = True, na = False), 'Injection'] = "Injection"
# Oral
df.loc[df['Interventions'].str.contains(oral1, case = False, na = False), 'Oral'] = "Oral"
df.loc[df['Interventions'].str.lower().str.lower().str.contains(oral2, case = True, na = False), 'Oral'] = "Oral"
# Topical
df.loc[df['Interventions'].str.contains(topical1, case = False, na = False), 'Topical'] = "Topical"
# Surgical
df.loc[df['Interventions'].str.contains(surgical1, case = False, na = False), 'Surgical'] = "Surgical"

## Brief Summary Column 
# Injection
df.loc[df['Brief Summary'].str.contains(injection1, case = False, na = False) & df['Injection'].isnull(),
        'Injection'] = "Injection"
df.loc[df['Brief Summary'].str.lower().str.lower().str.contains(injection2, case = True, na = False) & df['Injection'].isnull(),
        'Injection'] = "Injection"
# Oral
df.loc[df['Brief Summary'].str.contains(oral1, case = False, na = False) & df['Oral'].isnull() , 'Oral'] = "Oral"
df.loc[df['Brief Summary'].str.lower().str.lower().str.contains(oral2, case = True, na = False) & df['Oral'].isnull(),
        'Oral'] = "Oral"
# Topical
df.loc[df['Brief Summary'].str.contains(topical1, case = False, na = False) & df['Topical'].isnull()
       , 'Topical'] = "Topical"
# Surgical
df.loc[df['Brief Summary'].str.contains(surgical1, case = False, na = False) & df['Topical'].isnull(),
        'Surgical'] = "Surgical"


In [21]:
df = df.merge(my_intervention_methods[['nct_id', 'interv_method']], on = "nct_id", how = "left")
df = df.rename({"interv_method" : "Intervention_Route_List"}, axis = 1)

df["Intervention_Route_List"] = df[['Injection', 'Oral', 'Topical', 'Surgical']].apply(lambda row: [val for val in row if pd.notna(val) and val != "None"], axis=1)
df["Intervention_Route_List"] = df['Intervention_Route_List'].apply(lambda x: sorted(set(x)))
display(df['Intervention_Route_List'].value_counts(dropna = False))
display(df.shape)

# Fillna --> 'Other'
cond = (df['Intervention_Route_List'].apply(lambda x: isinstance(x, list) and len(x) == 0))

df.loc[cond, 'Intervention_Route_List'] = df.loc[cond, 'Intervention_Route_List'].apply(lambda x: ['Inter_Route_Other'])
display(df['Intervention_Route_List'].value_counts(dropna = False))

Intervention_Route_List
[]                                      28921
[Injection]                             17960
[Oral]                                   8320
[Topical]                                5732
[Injection, Surgical]                    3848
[Injection, Oral]                        3817
[Surgical]                               3705
[Injection, Topical]                     3412
[Oral, Topical]                           958
[Injection, Oral, Topical]                819
[Injection, Oral, Surgical]               659
[Oral, Surgical]                          572
[Injection, Surgical, Topical]            252
[Surgical, Topical]                       169
[Injection, Oral, Surgical, Topical]       22
[Oral, Surgical, Topical]                  11
Name: count, dtype: int64

(79177, 42)

Intervention_Route_List
[Inter_Route_Other]                     28921
[Injection]                             17960
[Oral]                                   8320
[Topical]                                5732
[Injection, Surgical]                    3848
[Injection, Oral]                        3817
[Surgical]                               3705
[Injection, Topical]                     3412
[Oral, Topical]                           958
[Injection, Oral, Topical]                819
[Injection, Oral, Surgical]               659
[Oral, Surgical]                          572
[Injection, Surgical, Topical]            252
[Surgical, Topical]                       169
[Injection, Oral, Surgical, Topical]       22
[Oral, Surgical, Topical]                  11
Name: count, dtype: int64

## Intervention_Route_Counts

In [22]:
df['Intervention_Route_Counts'] = df['Intervention_Route_List'].apply(lambda x: len(x))  
# Inter_Route_Other is a form of 'intervention' other than inj/oral/topical and must be counted.
df['Intervention_Route_Counts'].value_counts(dropna = False)

Intervention_Route_Counts
1    64638
2    12776
3     1741
4       22
Name: count, dtype: int64

## Placebo_Bin

In [23]:
my_placebo2 = my_placebo[my_placebo["group_type"] == "PLACEBO_COMPARATOR"]
df = df.merge(my_placebo2[["nct_id", "group_type"]], on = "nct_id", how = "left")
df = df.rename(columns = {"group_type" : "Placebo_Bin"})
display(df["Placebo_Bin"].value_counts(dropna = False))

df["Placebo_Bin"] = df["Placebo_Bin"].replace({np.NaN : "No", "PLACEBO_COMPARATOR" : "Yes"})
display(df["Placebo_Bin"].value_counts(dropna = False)) 


Placebo_Bin
NaN                   56385
PLACEBO_COMPARATOR    22792
Name: count, dtype: int64

Placebo_Bin
No     56385
Yes    22792
Name: count, dtype: int64

## Standard_Care_Bin

In [24]:
df = df.merge(my_soc[["nct_id", "Standard_Care"]], on = "nct_id", how = "left")
df = df.rename(columns={"Standard_Care" : "Standard_Care_Bin"})
display(df["Standard_Care_Bin"].value_counts(dropna=False))

df["Standard_Care_Bin"] = df["Standard_Care_Bin"].replace({np.NaN : "No"})
display(df["Standard_Care_Bin"].value_counts(dropna=False))


Standard_Care_Bin
NaN    76225
Yes     2952
Name: count, dtype: int64

Standard_Care_Bin
No     76225
Yes     2952
Name: count, dtype: int64

## Healthy_Bin

In [25]:
df = df.merge(my_eligibilities[["nct_id", "healthy_volunteers"]], on = "nct_id", how = "left")
df = df.rename(columns={"healthy_volunteers" : "Healthy_Bin"})
df['Healthy_Bin'] = df['Healthy_Bin'].replace({'Condition' : "No", "Healthy" : "Yes"})
display(df["Healthy_Bin"].value_counts(dropna = False))


Healthy_Bin
No     58087
Yes    21063
NaN       27
Name: count, dtype: int64

### NaN --> Drop

In [26]:
df = df.dropna(subset = ['Healthy_Bin'])
display(df["Healthy_Bin"].value_counts(dropna = False))


Healthy_Bin
No     58087
Yes    21063
Name: count, dtype: int64

## Covid_19_Bin

In [27]:
df = df.merge(my_covid[["nct_id", "mesh_term"]], on = "nct_id", how = "left")
df = df.rename(columns={"mesh_term" : "Covid_19_Bin"})
display(df["Covid_19_Bin"].value_counts(dropna=False))
df["Covid_19_Bin"] = df["Covid_19_Bin"].replace({np.NaN : "No", "COVID-19": "Yes"})
display(df["Covid_19_Bin"].value_counts(dropna=False))
display(df.shape)

Covid_19_Bin
NaN         77342
COVID-19     1808
Name: count, dtype: int64

Covid_19_Bin
No     77342
Yes     1808
Name: count, dtype: int64

(79150, 47)

## Conditions_Detail_List
- Source MEsH Ontologies and Drugs : https://meshb.nlm.nih.gov/treeView  
- Columns: my_conditions['name'] (list like) = df['Conditions'] (text like | seperated)


In [28]:
df = df.merge(my_conditions[["nct_id", "heading"]], on = "nct_id", how = "left") # my_conditions['name'] = df['Conditions]
df = df.rename(columns = {"heading" : "Conditions_Detail_List"})

df["Conditions_Detail_List"] = df["Conditions_Detail_List"].apply(lambda x: ["Cond_Detail_Other"] if isinstance(x, list)\
                                                               and len(x) == 0 else x)

display(Counter([elem for row_list in df["Conditions_Detail_List"] for elem in row_list]))

Counter({'Cond_Detail_Other': 18115,
         'Pathological Conditions, Signs and Symptoms': 16648,
         'Neoplasms': 15998,
         'Nervous System Diseases': 10507,
         'Respiratory Tract Diseases': 8521,
         'Immune System Diseases': 8370,
         'Skin and Connective Tissue Diseases': 7970,
         'Bacterial Infections and Mycoses': 7621,
         'Female Urogenital Diseases and Pregnancy Complications': 6906,
         'Digestive System Diseases': 6762,
         'Nutritional and Metabolic Diseases': 6719,
         'Cardiovascular Diseases': 6167,
         'Male Urogenital Diseases': 6076,
         'Virus Diseases': 5669,
         'Hemic and Lymphatic Diseases': 5654,
         'Endocrine System Diseases': 5248,
         'Mental Disorders': 5144,
         'Congenital, Hereditary, and Neonatal Diseases and Abnormalities': 3913,
         'Musculoskeletal and Neural Physiological Phenomena': 3271,
         'Musculoskeletal Diseases': 3193,
         'Psychological Pheno

In [29]:
# Merge low sample categries into more general ones based on the subject studyed.
cond_map = {
## Diseases/Disorders ==========================================================
         'Pathological Conditions, Signs and Symptoms': 'Pathological Conditions, Signs, Symptoms',   # Diseases

         'Nervous System Diseases': 'Nervous System',

         'Cond_Detail_Other': 'Cond_Detail_Other',

         'Neoplasms': 'Neoplasms', 

         'Cardiovascular Diseases': 'Cardiovascular',  

         'Skin and Connective Tissue Diseases': 'Skin, Tissue', 
         'Tissues': 'Skin, Tissue', 

         'Respiratory Tract Diseases': 'Respiratory, Circulatory', 
         'Circulatory and Respiratory Physiological Phenomena': 'Respiratory, Circulatory', 

         'Female Urogenital Diseases and Pregnancy Complications': 'Urogenital, Pregnancy', 
         'Male Urogenital Diseases': 'Urogenital, Pregnancy', 
         'Reproductive and Urinary Physiological Phenomena': 'Urogenital, Pregnancy',  
         
         'Bacterial Infections and Mycoses': 'Bacterial Infections and Mycoses', 
         'Virus Diseases': 'Virus Diseases', 

         'Immune System Diseases': 'Hemic, Lymphatic, Immune System', 
         'Immune System Phenomena': 'Hemic, Lymphatic, Immune System', 
         'Hemic and Immune Systems': 'Hemic, Lymphatic, Immune System', 
         'Hemic and Lymphatic Diseases': 'Hemic, Lymphatic, Immune System', 
         
         'Digestive System Diseases': 'Digestive System, Nutritional, Metabolic', 
         'Nutritional and Metabolic Diseases': 'Digestive System, Nutritional, Metabolic', 
                  
         'Musculoskeletal Diseases': 'Musculoskeletal, Neural', 
         'Musculoskeletal and Neural Physiological Phenomena': 'Musculoskeletal, Neural', 

         'Endocrine System Diseases': 'Endocrine System', 

         'Congenital, Hereditary, and Neonatal Diseases and Abnormalities': 'Hereditary, Neonatal, Abnormalities', 

         'Wounds and Injuries': 'Wounds, Injuries', 

         'Eye Diseases': 'Eye', 

         'Stomatognathic Diseases': 'Stomatognathic, Otorhinolaryngologic', 
         'Otorhinolaryngologic Diseases': 'Stomatognathic, Otorhinolaryngologic', 

         'Chemically-Induced Disorders': 'Chemical Disorders',  # Diseases

# 'Psychiatry, Psychology, Anthropology, Sociology'
         'Psychological Phenomena and Processes': 'Psychiatry, Psychology, Anthropology, Sociology', 
         'Behavior and Behavior Mechanisms': 'Psychiatry, Psychology, Anthropology, Sociology', 
         'Mental Disorders': 'Psychiatry, Psychology, Anthropology, Sociology', 
         'Social Sciences': 'Psychiatry, Psychology, Anthropology, Sociology',   # Anthropology, Sociology
         'Human Activities': 'Psychiatry, Psychology, Anthropology, Sociology',  # Anthropology, Sociology
         'Education' : 'Psychiatry, Psychology, Anthropology, Sociology',

# 'Phenomena, Processes'
         'Physiological Phenomena': 'Phenomena, Processes', 
         'Biological Phenomena': 'Phenomena, Processes', 
         'Physical Phenomena': 'Phenomena, Processes', 
         'Genetic Phenomena': 'Phenomena, Processes', 
         'Cell Physiological Phenomena': 'Phenomena, Processes', 
         'Cells': 'Phenomena, Processes',  # Anatomy
         'Fluids and Secretions': 'Phenomena, Processes',  # Anatomy 
         'Chemical Actions and Uses': 'Phenomena, Processes',  # Chemicals and Drugs

# Health Care, Environment, Diagnostic, Equipment, Other
         'Health Care Quality, Access, and Evaluation': 'Health Care, Therapeutics', 
         'Health Care Economics and Organizations' : 'Health Care, Therapeutics', 
         'Health Services Administration': 'Health Care, Therapeutics', 
         'Health Care Facilities, Manpower, and Services': 'Health Care, Therapeutics', 
         'Environment and Public Health': 'Health Care, Therapeutics', 
         'Information Science' : 'Health Care, Therapeutics',  
         'Diagnosis': 'Health Care, Therapeutics',    
         'Investigative Techniques': 'Health Care, Therapeutics',  
         'Therapeutics': 'Health Care, Therapeutics', 
}



display(Counter([elem for row_list in df["Conditions_Detail_List"] for elem in row_list]))
df['Conditions_Detail_List'] = df['Conditions_Detail_List'].apply(lambda row: [cond_map.get(val, val) for val in row])

display(Counter([elem for row_list in df["Conditions_Detail_List"] for elem in row_list]))

Counter({'Cond_Detail_Other': 18115,
         'Pathological Conditions, Signs and Symptoms': 16648,
         'Neoplasms': 15998,
         'Nervous System Diseases': 10507,
         'Respiratory Tract Diseases': 8521,
         'Immune System Diseases': 8370,
         'Skin and Connective Tissue Diseases': 7970,
         'Bacterial Infections and Mycoses': 7621,
         'Female Urogenital Diseases and Pregnancy Complications': 6906,
         'Digestive System Diseases': 6762,
         'Nutritional and Metabolic Diseases': 6719,
         'Cardiovascular Diseases': 6167,
         'Male Urogenital Diseases': 6076,
         'Virus Diseases': 5669,
         'Hemic and Lymphatic Diseases': 5654,
         'Endocrine System Diseases': 5248,
         'Mental Disorders': 5144,
         'Congenital, Hereditary, and Neonatal Diseases and Abnormalities': 3913,
         'Musculoskeletal and Neural Physiological Phenomena': 3271,
         'Musculoskeletal Diseases': 3193,
         'Psychological Pheno

Counter({'Cond_Detail_Other': 18115,
         'Pathological Conditions, Signs, Symptoms': 16648,
         'Neoplasms': 15998,
         'Hemic, Lymphatic, Immune System': 14046,
         'Digestive System, Nutritional, Metabolic': 13481,
         'Urogenital, Pregnancy': 13070,
         'Psychiatry, Psychology, Anthropology, Sociology': 10981,
         'Nervous System': 10507,
         'Respiratory, Circulatory': 8570,
         'Skin, Tissue': 8062,
         'Bacterial Infections and Mycoses': 7621,
         'Musculoskeletal, Neural': 6464,
         'Cardiovascular': 6167,
         'Virus Diseases': 5669,
         'Endocrine System': 5248,
         'Hereditary, Neonatal, Abnormalities': 3913,
         'Health Care, Therapeutics': 2417,
         'Stomatognathic, Otorhinolaryngologic': 2187,
         'Eye': 2098,
         'Phenomena, Processes': 1983,
         'Wounds, Injuries': 1294,
         'Chemical Disorders': 1215})

In [30]:
# Search for Condition/body part worlds in 'Conditions' Column if 
Conditions2 = pd.DataFrame(sorted(set('|'.join(df.loc[
        df['Conditions_Detail_List'].apply(lambda row: (
            'Phenomena, Processes' in row or 'Pathological Conditions, Signs, Symptoms' in row or 'Cond_Detail_Other' in row)), 
            'Conditions'].dropna().astype(str)).replace(', ', '|').split('|'))), columns = ['Conditions2'])

# Split Conditions2 into 4 columns --> Keep only the first 4 columns
Conditions2 = Conditions2['Conditions2'].str.split(' ', expand = True)
Conditions2 = Conditions2.iloc[:, 0:4]
Conditions2 = Conditions2.drop_duplicates(keep = 'first').dropna()
# Keep worlds of Conditions/Body parts as a list.
cond2 = sorted(set(Conditions2[0].tolist())) + sorted(set(Conditions2[1].tolist())) +\
     sorted(set(Conditions2[2].tolist())) + sorted(set(Conditions2[3].tolist()))


#### NaNs+ --> Fillna text -> Other

In [31]:
# Create list of key worlds to search in 'Conditions' to fill Conditions_List/Conditions_Detail_List columns
texts = [
 # a   
 'abdominal','abdominis','abnormality','abortion','accident','acne','activities','adrenal','aging','airway','allergy',
 'alveolar','alzheimer','anaesthesia','analgesi','analgesia','anemia','anesthesia','anorexia','antibiotic','antibody mediated rejection',
 'anticalculus','anticoag','antimicrobial resistance','anxiety','aortic','apnoea','appetite','arthritis','asthma','atheroscler',
 'atherosclerotic','attention deficit hyperactivity disorder','autism','avascular','axillary',
 # b
 'bacteria','bacterial','behavior',"bichat's",
 'bilateral','bile','bioavailability','bioequivalence','bipolar','bladder','bleeding','blood pressure','blood vessels','bone','bones',
 'botulinum','bowel','bradycardia','brain','breast','bronchitis','bronchoscopic','buprenorphine','bypass', 'blood sugar',
 # c
 'cancer','cannabis','carcinoma',
 'cardiac','cardiovascular','health care','caregiver','carotid','cartilage','cell','cellulite','cerebral','cervical','cervix','cesarean',
 'changas','cheek','chemical','chemotherapy','chest','cholesterol','chromosome-negative','cigar','clostridium','coagulation','cocaine',
 'coffee','colitis','colon','colonoscopy','connective tissue','constipation','contraception','corneae','coronary','covid','critical ill',
 'critically ill','cruciate ligament','cystic fibrosis',
 # d
 'defecation','dementia','dental','depression','dermatitis','dexamethasone-induced','diabet',
 'diabetes','diabetic','diaphragm','diarrhea','diet','digestive','donor','drug effect','drug-induced','dyslipidemia','dyspepsia',
 # e
 'ear','eczema','education','encephalitis','endocrine','endometriosis','epilepsy','epithelial','esophageal','esophagus','eustachian',
 'exercise','extracorporeal','eye','fat','fever','fevers','fibrosclerotic','fibrosis/tubular','food','foot',
 # g
 'gallbladder','gastric','gastritis','gastro','gastroesophageal','gastrointestinal','gastroretentive','genetic','genitourinary','gingival',
 'glaucoma','gout',
 # h
 'haemopoietic','hamstring','hand','healthcare','heart','heart failure','helicobacter pylori','helminth','hematopoietic','hemmorhagic',
 'hemodialysis','hemorrhage','hepatic','hepatics','hepatitis','hernia','heroin','hiccup','hip','hormones','hypertension','hypertensive',
 'hypothermia','hypovolaemia','hypovolemia','hypoxia','hysterectomy',
 # i
 'ileostomy','immun','immune','incisional','incretin',
 'infant','infected','infection','inflammation','inflammatory bowel','inguinal','insomnia','integumentary','interstitial','intestinal',
 'intestine','intra-cytoplasmic','intracellular','iris','ischemia','ischemic','jaw',
 # k
 'kidney','knee',
 # l
 'labour','laryngeal','leg','leukemia','ligament','limb','limbus','lipid','liver','lumbar','lung','lupus','lymph nodes','lymphatic','lymphoid',
 # m
 'malaria','masseter','melatonin','menopausal','menopause','menstrual','mental','metabolism','metastatic','methamphetamin',
 'migraine','mouth','mucous','muscle','muscles','muscular','musculoskeletal system','mutations','myalgia','myomectomy',
 # n
 'nasal','nausea','neck','neoplasm','nephritis','nerve','nerves','nervous system','neuromuscular','neuropsychiatric','neurotrophic',
 'nicotin','nurse','nutri','nutrient',
 # o
 'obese','obesity','occlusive','ocular','oocyte','opioid','osteo','ovaries','oxygen','oxygenation',
 # p
 'pain','pancrea','pancreas','panniculopathy','papillomavirus','papiloma','parathyroid','parkinson','pelvic','pelvis','penis','perianal',
 'pghysical activity','pharmacokinetic','pharmacokinetics','phycolog','physical','physical activity','plasma','polyp','popliteal',
 'post-lung','postoperative','postprandial','pregnan','prenatal','prolapse','prostate','psoriasis','pulmonary',
 #r
 'radio','radiotherapy-induced',
 'rectal','rectum','renal','reproductive','respiratory', r'\brib\b', r'\bsars\b',
 # s
 'scalp','schizophrenia','scoliosis','sedation','sepsis','shoulder','skeletal','skin','sleep','smok','social','spinal',
 'spine','spleen','staphylococcus','stoma','stomach','streptococcus','stress','stroke','sympathetic',
 # t
 'tachycardia','teeth',
 'temporomandibular','tendon','tendons','testes','testicles','tetrahydrocannabinol','thoracic','thymus','thyroid','tissue',
 'tobacco','tongue','tonsillitis','tooth','toxin-induced','trachea','tracheo','tuberculosis','tumor',
 # u
 'ulceration','unilateral','ureter','urethra','urin','urinary','urine','uter','uterovaginal','uterus','vagina','vaginal','valgus',
 # v
 'vaping','vascular','vasomotor','ventral','verrucous','violence','vision','visual','vitamin','vomiting','vulvar',
 # w
 'white spot lesion','wound','wrinkle', 'weight gain']

texts = sorted(set([t.lower() for t in texts]))


In [32]:
display(df["Conditions_Detail_List"].apply(lambda x : x == ['Cond_Detail_Other']).sum())

# ===============================================================================================================================
# Covid-19 Column
df.loc[(df['Covid_19_Bin'] == 'Yes') & (df['Conditions_Detail_List'].apply(lambda x: x == ['Cond_Detail_Other'])),'Conditions_Detail_List'] = df.loc[df['Covid_19_Bin'] == 'Yes','Conditions_Detail_List'].apply(
    lambda x: ['Respiratory, Circulatory', 'Virus Diseases']) # Top list element.

# ===============================================================================================================================
# Conditions Column
for text in texts:
    case = False
    try:
        # Constraints
        constr2 = ((df["Conditions"].str.contains(text, case = case)) & (df['Conditions_Detail_List'].apply(lambda x: x == ['Cond_Detail_Other'])))
        # text_new
        text_new2 = df.loc[df['Conditions'].str.contains(text, case = case) & df['Conditions_Detail_List'].apply(
            lambda x: x != ['Cond_Detail_Other'])]['Conditions_Detail_List'].value_counts(ascending = False).index[0] 
        # Applying the changes
        df.loc[constr2, 'Conditions_Detail_List'] = df.loc[constr2, 'Conditions_Detail_List'].apply(lambda x: text_new2)
    
    except IndexError:
        continue

# ===============================================================================================================================
texts = r'\bALS\b|\bBPH\b|\bACL\b|\bALL\b|\bAML\b|\bARDS\b|\bAccidental Falls\b|\bAlcohol\b|\bHPV\b|\
    \bCOPD\b|\bPTSD\b|\bNSCLC\b|\bPCOS\b|\bOHSS\b|\bOSA\b|\bNAFLD\b|\bPOH\b|\bABPA\b|\bIBD\b|\
    \bPACU\b|\bPIK3CA\b|\bAOD\b|\bBPF\b\bNYHA\b|\bIPOM\b|\bHIV\b|\baids\b|\bAIDS\b|\badhd\b'

case = True
try:
# Constraints
    constr2 = ((df["Conditions"].str.contains(texts, case = case)) & (df['Conditions_Detail_List'].apply(lambda x: x == ['Cond_Detail_Other'])))
# text_new - 1st choice as comments = for most common list description
    # text_new2 = df.loc[df['Conditions'].str.contains(texts, case = case) & df['Conditions_Detail_List'].apply(lambda x: x != ['Cond_Detail_Other'])]\
    # ['Conditions_Detail_List'].value_counts(ascending = False).index[0]
    
# text_new - 1st choice as comments = for most common element description
    all_elements2 = [elem for sublist in df.loc[df['Conditions'].str.contains(texts, case=case) & df['Conditions_Detail_List'].apply(lambda x: x != ['Cond_Detail_Other']), 'Conditions_Detail_List'] for elem in sublist]
    text_new2 = [Counter(all_elements2).most_common(1)[0][0]]
    # Applying the changes
    df.loc[constr2, 'Conditions_Detail_List'] = df.loc[constr2, 'Conditions_Detail_List'].apply(lambda x: text_new2)

except IndexError:
    pass

# ===============================================================================================================================
# Erase 'Phenomena, Processes' if len_list > 1. They are mostly phenomena due to Conditions. 
'''
df['Conditions_Detail_List'] = df['Conditions_Detail_List'].apply(
    lambda row: [elem for elem in row if elem != 'Phenomena, Processes'] if 
    ('Phenomena, Processes' in row and len(row) >= 2) else row)'''
'''
# Erase 'Pathological Conditions, Signs, Symptoms' if len_list > 1. They are mostly symptoms due to Conditions. 
df['Conditions_Detail_List'] = df['Conditions_Detail_List'].apply(
    lambda row: [elem for elem in row if elem != 'Pathological Conditions, Signs, Symptoms'] if 
    ('Pathological Conditions, Signs, Symptoms' in row and len(row) >= 2) else row)'''

# ===============================================================================================================================
display(df["Conditions_Detail_List"].apply(lambda x : x == ['Cond_Detail_Other']).sum())

# Sort/Set
df["Conditions_Detail_List"] = df["Conditions_Detail_List"].apply(lambda x: sorted(set(x)))
display(df["Conditions_Detail_List"].value_counts(dropna = False))

18115

8584

Conditions_Detail_List
[Cond_Detail_Other]                                                                                                                                                         8584
[Neoplasms, Skin, Tissue]                                                                                                                                                   2760
[Musculoskeletal, Neural, Nervous System, Pathological Conditions, Signs, Symptoms, Psychiatry, Psychology, Anthropology, Sociology]                                        2738
[Bacterial Infections and Mycoses, Respiratory, Circulatory, Virus Diseases]                                                                                                2721
[Psychiatry, Psychology, Anthropology, Sociology]                                                                                                                           2526
                                                                                            

## Adverse_List

In [33]:
df = df.merge(my_adverse[["nct_id", "event_type", "event_sum"]], on = "nct_id", how = "left")
df = df.rename(columns={"event_type" : "Adverse_List"})
display(df["Adverse_List"].value_counts(dropna=False))

Adverse_List
[]                         52492
[Death, Other, Serious]    11511
[Other, Serious]            6138
[Other]                     5064
[Death, Other]              2089
[Death]                      991
[Death, Serious]             532
[Serious]                    333
Name: count, dtype: int64

### NaNs++ -> None

In [34]:
# if NaN here means no adverse recorded!
df["Adverse_List"] = df["Adverse_List"].apply(lambda x: ['None'] if (isinstance(x, list) and len(x) == 0) else x) 

# Unspecified instead of Other because in drop_first for dummy encoing words like other/none are dropped, and this column
# already has a none category.
df["Adverse_List"] = df["Adverse_List"].apply(lambda row: ['Unspecified' if x == 'Other' else x for x in row])
df['Adverse_List'] = df['Adverse_List'].apply(lambda row: ['Adv_' + elem for elem in row])

display(df["Adverse_List"].value_counts(dropna=False))

Adverse_List
[Adv_None]                                   52492
[Adv_Death, Adv_Unspecified, Adv_Serious]    11511
[Adv_Unspecified, Adv_Serious]                6138
[Adv_Unspecified]                             5064
[Adv_Death, Adv_Unspecified]                  2089
[Adv_Death]                                    991
[Adv_Death, Adv_Serious]                       532
[Adv_Serious]                                  333
Name: count, dtype: int64

## Adverse_Counts
- number of adverse noticed at the participants is not the same with non None adverse events noticed above. 
- Distribution of Adverse_Counts is up to 75% at 0. Only Adverse_Bin is going to be used.

In [35]:
df = df.rename(columns={"event_sum" : "Adverse_Counts"}) 
df["Adverse_Counts"] = df["Adverse_Counts"].astype(int)
display(df["Adverse_Counts"].value_counts(dropna = False))

Adverse_Counts
0       53326
1        1342
3         897
6         635
4         531
        ...  
1622        1
2739        1
2629        1
996         1
704         1
Name: count, Length: 1507, dtype: int64

### NaNs
Some rows have Adverse_Counts == 0 however adverse_list == ['Adv_Death']. But we do not know the number of events

In [36]:
df.loc[(df['Adverse_Counts'] == 0) & (df['Adverse_List'].apply(lambda x: x == ['Adv_Death'])), 'Adverse_Counts'].shape[0]

834

## Adverse_Counts_Log
- Distribution of Adverse_Counts is up to 75% at 0. Only Adverse_Bin is going to be used.

In [37]:
df['Adverse_Counts_Log'] = np.log1p(df['Adverse_Counts'])
display(df['Adverse_Counts_Log'].describe())

count    79150.000000
mean         1.191107
std          1.948227
min          0.000000
25%          0.000000
50%          0.000000
75%          2.397895
max         11.159759
Name: Adverse_Counts_Log, dtype: float64

## Adverse_Bin

In [38]:
df.loc[df["Adverse_Counts"] == 0, "Adverse_Bin"] = 'No'
df.loc[df["Adverse_Counts"] >= 1, "Adverse_Bin"] = 'Yes'

## Adverse_System_List

In [39]:
df = df.merge(my_adverse_system[["nct_id", "organ_system"]], on = "nct_id", how = "left")
df = df.rename(columns={"organ_system" : "Adverse_System_List"})
display(Counter([elem for row_list in df["Adverse_System_List"].dropna() for elem in row_list]))  

Counter({'Gastrointestinal': 20619,
         'Nervous System': 19514,
         'General': 18888,
         'Infections': 18405,
         'Respiratory': 16590,
         'Musculoskeletal': 15763,
         'Skin': 14705,
         'Injury, Poisoning, Procedural': 13516,
         'Product Issues, Investigations': 13197,
         'Metabolism/ Nutrition': 12817,
         'Cardio': 12777,
         'Psychiatric, Social': 12219,
         'Blood, Lymphatic, Endocrine': 12200,
         'Vascular': 12185,
         'Renal, Urinary': 11375,
         'Eye': 8933,
         'Neoplasms': 8149,
         'Reproductive, Pregnancy, Breast': 7169,
         'Hepatobiliary': 7144,
         'Ear': 5862,
         'Immune System': 5402,
         'Surgical, Medical': 2724})

### NaNs++ -> None
- Number is cose to Adverse_List ~ 43 thousands missing

In [41]:
df[df['Adverse_System_List'].apply(lambda x: x == ['General'])][['Adverse_List', 'Adverse_System_List' ]]

,Adverse_List,Adverse_System_List
41,"[Adv_Death, Adv_Serious]",[General]
262,"[Adv_Death, Adv_Unspecified]",[General]
534,"[Adv_Death, Adv_Unspecified, Adv_Serious]",[General]
639,[Adv_Unspecified],[General]
1116,[Adv_Unspecified],[General]
...,...,...
77795,"[Adv_Death, Adv_Serious]",[General]
78006,[Adv_Unspecified],[General]
78379,"[Adv_Death, Adv_Unspecified, Adv_Serious]",[General]
78439,"[Adv_Death, Adv_Serious]",[General]


In [42]:
# Checked: Only None and Death at Adv_List col for nan in Adverse_System_List col
cond1 = (df['Adverse_System_List'].isnull()) & (df['Adverse_List'].apply(lambda x: x == ['Adv_None'])) 
cond2 = (df['Adverse_System_List'].isnull()) & (df['Adverse_List'].apply(lambda x: x == ['Adv_Death']))

df['Adverse_System_List'] = np.where(cond1, 'None', df['Adverse_System_List'])
df['Adverse_System_List'] = np.where(cond2, 'General', df['Adverse_System_List'])
df['Adverse_System_List'] = df['Adverse_System_List'].apply(lambda x: [x] if isinstance(x, str) else x)

df["Adverse_System_List"] = df["Adverse_System_List"].apply(lambda x: sorted(set(x)) if isinstance(x, str) else x)
df['Adverse_System_List'] = df['Adverse_System_List'].apply(lambda lst: ['Adv_Syst_' + elem for elem in lst])

display(df["Adverse_System_List"].explode().value_counts(dropna=False))
display(Counter([elem for row_list in df["Adverse_System_List"] for elem in row_list]))


Adverse_System_List
Adv_Syst_None                               52472
Adv_Syst_Gastrointestinal                   20619
Adv_Syst_General                            19866
Adv_Syst_Nervous System                     19514
Adv_Syst_Infections                         18405
Adv_Syst_Respiratory                        16590
Adv_Syst_Musculoskeletal                    15763
Adv_Syst_Skin                               14705
Adv_Syst_Injury, Poisoning, Procedural      13516
Adv_Syst_Product Issues, Investigations     13197
Adv_Syst_Metabolism/ Nutrition              12817
Adv_Syst_Cardio                             12777
Adv_Syst_Psychiatric, Social                12219
Adv_Syst_Blood, Lymphatic, Endocrine        12200
Adv_Syst_Vascular                           12185
Adv_Syst_Renal, Urinary                     11375
Adv_Syst_Eye                                 8933
Adv_Syst_Neoplasms                           8149
Adv_Syst_Reproductive, Pregnancy, Breast     7169
Adv_Syst_Hepatobiliary        

Counter({'Adv_Syst_None': 52472,
         'Adv_Syst_Gastrointestinal': 20619,
         'Adv_Syst_General': 19866,
         'Adv_Syst_Nervous System': 19514,
         'Adv_Syst_Infections': 18405,
         'Adv_Syst_Respiratory': 16590,
         'Adv_Syst_Musculoskeletal': 15763,
         'Adv_Syst_Skin': 14705,
         'Adv_Syst_Injury, Poisoning, Procedural': 13516,
         'Adv_Syst_Product Issues, Investigations': 13197,
         'Adv_Syst_Metabolism/ Nutrition': 12817,
         'Adv_Syst_Cardio': 12777,
         'Adv_Syst_Psychiatric, Social': 12219,
         'Adv_Syst_Blood, Lymphatic, Endocrine': 12200,
         'Adv_Syst_Vascular': 12185,
         'Adv_Syst_Renal, Urinary': 11375,
         'Adv_Syst_Eye': 8933,
         'Adv_Syst_Neoplasms': 8149,
         'Adv_Syst_Reproductive, Pregnancy, Breast': 7169,
         'Adv_Syst_Hepatobiliary': 7144,
         'Adv_Syst_Ear': 5862,
         'Adv_Syst_Immune System': 5402,
         'Adv_Syst_Surgical, Medical': 2724})

## Adverse_System_Counts
- How many organ systems where affected

In [43]:
df['Adverse_System_Counts'] = df['Adverse_System_List'].apply(
    lambda x: len(x) if isinstance(x, list) and x != ['Adv_Syst_None'] else 0)
display(df['Adverse_System_Counts'].value_counts(dropna = False))

Adverse_System_Counts
0     52472
1      2857
2      1513
3      1342
4      1284
18     1257
17     1222
6      1217
5      1212
8      1205
19     1192
7      1183
9      1155
16     1147
12     1144
11     1123
10     1099
15     1094
13     1063
14     1054
20     1046
21      886
22      383
Name: count, dtype: int64

## Adverse_System_Counts_Log

In [44]:
df['Adverse_System_Counts_Log'] = np.log1p(df["Adverse_System_Counts"]) 
df['Adverse_System_Counts_Log'].describe()

count    79150.000000
mean         0.734141
std          1.119866
min          0.000000
25%          0.000000
50%          0.000000
75%          1.609438
max          3.135494
Name: Adverse_System_Counts_Log, dtype: float64

## Study Design

Source: 
https://clinicaltrials.gov/policy/protocol-definitions#study-design

Clinical Trials.gov Documentation: 
1. Allocation: 
The method by which participants are assigned to arms in a clinical trial.
- N/A (not applicable): For a single-arm trial
- Randomized: Participants are assigned to intervention groups by chance
- Nonrandomized: Participants are expressly assigned to intervention groups through a non-random method, such as physician choice 
2. Intervention_model (Single Group, Crossover etc.)
3. Masking  (Blindness, None, Double etc.)
4. Primary purpose (Treatment, Screening etc.)



### Allocation_List

In [45]:
df = df.merge(my_designs[["nct_id", "allocation"]], on = "nct_id", how = "left")
df = df.rename(columns = {"allocation" : "Allocation_List"})
display(df["Allocation_List"].value_counts(dropna=False))
display(df.shape)

Allocation_List
RANDOMIZED        51573
NA                18811
NON_RANDOMIZED     8522
None                244
Name: count, dtype: int64

(79150, 56)

#### NaN --> Fillna text + Drop

In [46]:
# Allocation Fillna
df.loc[(df["Allocation_List"].isnull()) & (df["Brief Summary"].str.contains(r"(?i)\snon.randomized\s|open label" , case = False , na = False) ) , "Allocation_List"] = "NON_RANDOMIZED"
df.loc[(df["Allocation_List"].isnull()) & (df["Brief Summary"].str.contains(r"(?i)\srandomized\s")) & ~(df["Brief Summary"].str.contains(r"(?i)\snon.randomized\s", case = False , na = False)) , "Allocation_List"] = "RANDOMIZED"

df["Allocation_List"] = df["Allocation_List"].replace({"NA" : "NA_RANDOMIZED"})
display(df["Allocation_List"].value_counts(dropna = False))
df = df.dropna(subset = ["Allocation_List"])
display(df["Allocation_List"].value_counts(dropna = False))


Allocation_List
RANDOMIZED        51580
NA_RANDOMIZED     18811
NON_RANDOMIZED     8538
None                221
Name: count, dtype: int64

Allocation_List
RANDOMIZED        51580
NA_RANDOMIZED     18811
NON_RANDOMIZED     8538
Name: count, dtype: int64

### Intervention_Model_List

In [47]:
df = df.merge(my_designs[["nct_id", "intervention_model"]], on = "nct_id", how = "left")
df = df.rename(columns={"intervention_model" : "Intervention_Model_List"})
display(df["Intervention_Model_List"].value_counts(dropna=False))

Intervention_Model_List
PARALLEL        45258
SINGLE_GROUP    21305
CROSSOVER        7721
SEQUENTIAL       3920
FACTORIAL         634
None               91
Name: count, dtype: int64

#### NaNs -> Fillna text + Drop

In [48]:
# Fillna Intervention model
# display(df.loc[df["Intervention_Model"].isnull() & df["Brief Summary"].str.contains(r"(?i)PARALLEL|SINGLE GROUP|CROSSOVER|SEQUENTIAL|FACTORIAL"), "Brief Summary"].values[0:2])

df.loc[df["Intervention_Model_List"].isnull() & df["Brief Summary"].str.contains(r"PARALLEL", case = False), "Intervention_Model_List"] = "PARALLEL"
df.loc[df["Intervention_Model_List"].isnull() & df["Brief Summary"].str.contains(r"SINGLE GROUP", case = False), "Intervention_Model_List"] = "SINGLE_GROUP"
df.loc[df["Intervention_Model_List"].isnull() & df["Brief Summary"].str.contains(r"CROSSOVER", case = False), "Intervention_Model_List"] = "CROSSOVER"
df.loc[df["Intervention_Model_List"].isnull() & df["Brief Summary"].str.contains(r"SEQUENTIAL", case = False), "Intervention_Model_List"] = "SEQUENTIAL"
df.loc[df["Intervention_Model_List"].isnull() & df["Brief Summary"].str.contains(r"FACTORIAL", case = False), "Intervention_Model_List"] = "FACTORIAL"

display(df["Intervention_Model_List"].value_counts(dropna=False))
df = df.dropna(subset = ["Intervention_Model_List"])
display(df["Intervention_Model_List"].value_counts(dropna=False))
display(df.shape)

Intervention_Model_List
PARALLEL        45260
SINGLE_GROUP    21305
CROSSOVER        7722
SEQUENTIAL       3925
FACTORIAL         634
None               83
Name: count, dtype: int64

Intervention_Model_List
PARALLEL        45260
SINGLE_GROUP    21305
CROSSOVER        7722
SEQUENTIAL       3925
FACTORIAL         634
Name: count, dtype: int64

(78846, 57)

### Masking_List

ClinicalTrials.gov Documentation:
•	Masking
A clinical trial design strategy in which one or more parties involved in the trial, such as the investigator or participants, do not know which participants have been assigned which interventions. Types of masking include: open label, single blind masking, and double-blind masking.


In [49]:
df = df.merge(my_designs[["nct_id", "masking"]], on = "nct_id", how = "left")
df = df.rename(columns = {"masking" : "Masking_List"})
df['Masking_List'] = df['Masking_List'].replace('NONE', 'MASK_NONE')
display(df["Masking_List"].value_counts(dropna=False))

Masking_List
MASK_NONE    41476
QUADRUPLE    13694
DOUBLE       11173
TRIPLE        7197
SINGLE        5241
None            65
Name: count, dtype: int64

#### NaN --> Fillna text + Drop

In [50]:
# Fillna from Brief Summary 
df.loc[df["Masking_List"].isnull() & df["Brief Summary"].str.contains(r"(?i)OPEN.LABEL") , "Masking_List"] = "MASK_NONE"
df.loc[df["Masking_List"].isnull() & df["Brief Summary"].str.contains(r"(?i)SINGLE.BLIND") , "Masking_List"] = "SINGLE"
df.loc[df["Masking_List"].isnull() & df["Brief Summary"].str.contains(r"(?i)DOUBLE.BLIND") , "Masking_List"] = "DOUBLE"
df.loc[df["Masking_List"].isnull() & df["Brief Summary"].str.contains(r"(?i)TRIPLE.BLIND") , "Masking_List"] = "TRIPLE"
df.loc[df["Masking_List"].isnull() & df["Brief Summary"].str.contains(r"(?i)QUADRUPLE.BLIND") , "Masking_List"] = "QUADRUPLE"

display(df["Masking_List"].value_counts(dropna = False))
df = df.dropna(subset = ["Masking_List"])
display(df["Masking_List"].value_counts(dropna = False))
display(df.shape)

Masking_List
MASK_NONE    41485
QUADRUPLE    13694
DOUBLE       11175
TRIPLE        7197
SINGLE        5241
None            54
Name: count, dtype: int64

Masking_List
MASK_NONE    41485
QUADRUPLE    13694
DOUBLE       11175
TRIPLE        7197
SINGLE        5241
Name: count, dtype: int64

(78792, 58)

### Masking_Detail_List

In [51]:
df = df.merge(my_designs[["nct_id", "masking_detail"]], on = "nct_id", how = "left")
df = df.rename(columns = {"masking_detail" : "Masking_Detail_List"})

#### NaN --> Fillna + Drop

In [52]:
# Fillna Masking_Detail
# Masking = 'NONE' --> Masking_Details = ['NONE']
'''display(df.loc[df["Masking_Detail_List"].apply(    # value_counts for list element columns
    lambda x: isinstance(x, list) and len(x) == 0)]["Masking_List"].value_counts(dropna=False))'''
# For other levels (single, double etc.) no standard person is blinded.
'''display(df.loc[df["Masking_List"] == 'SINGLE'][["Masking_Detail_List", "Masking_List"]])'''

df.loc[(df["Masking_Detail_List"].apply(
    lambda x: isinstance(x, list) and len(x) == 0)) & \
        (df["Masking_List"] == "MASK_NONE"), "Masking_Detail_List"] = "MASK_DET_NONE"

df["Masking_Detail_List"] = df["Masking_Detail_List"].apply(lambda x: ["MASK_DET_NONE"] if x == "MASK_DET_NONE" else x)
df["Masking_Detail_List"] = df["Masking_Detail_List"].apply(lambda x: np.NaN if isinstance(x, list) and len(x) == 0 else x)

display(df["Masking_Detail_List"].value_counts(dropna=False))
df = df.dropna(subset = ["Masking_Detail_List"])
display(df["Masking_Detail_List"].value_counts(dropna=False))
display(df.shape)

Masking_Detail_List
[MASK_DET_NONE]                                                  41485
[PARTICIPANT, INVESTIGATOR, CARE_PROVIDER, OUTCOMES_ASSESSOR]    13694
[INVESTIGATOR, PARTICIPANT]                                       8711
[INVESTIGATOR, CARE_PROVIDER, PARTICIPANT]                        3868
[PARTICIPANT, INVESTIGATOR, OUTCOMES_ASSESSOR]                    2700
[PARTICIPANT]                                                     2390
[OUTCOMES_ASSESSOR]                                               2048
[PARTICIPANT, OUTCOMES_ASSESSOR]                                  1072
[PARTICIPANT, CARE_PROVIDER]                                       762
[INVESTIGATOR]                                                     680
[PARTICIPANT, CARE_PROVIDER, OUTCOMES_ASSESSOR]                    532
[INVESTIGATOR, OUTCOMES_ASSESSOR]                                  345
NaN                                                                181
[CARE_PROVIDER]                                          

Masking_Detail_List
[MASK_DET_NONE]                                                  41485
[PARTICIPANT, INVESTIGATOR, CARE_PROVIDER, OUTCOMES_ASSESSOR]    13694
[INVESTIGATOR, PARTICIPANT]                                       8711
[INVESTIGATOR, CARE_PROVIDER, PARTICIPANT]                        3868
[PARTICIPANT, INVESTIGATOR, OUTCOMES_ASSESSOR]                    2700
[PARTICIPANT]                                                     2390
[OUTCOMES_ASSESSOR]                                               2048
[PARTICIPANT, OUTCOMES_ASSESSOR]                                  1072
[PARTICIPANT, CARE_PROVIDER]                                       762
[INVESTIGATOR]                                                     680
[PARTICIPANT, CARE_PROVIDER, OUTCOMES_ASSESSOR]                    532
[INVESTIGATOR, OUTCOMES_ASSESSOR]                                  345
[CARE_PROVIDER]                                                     98
[INVESTIGATOR, CARE_PROVIDER, OUTCOMES_ASSESSOR]         

(78611, 59)

In [53]:
df['Masking_Detail_List'].apply(lambda x: sorted(set(x))).value_counts()

Masking_Detail_List
[MASK_DET_NONE]                                                  41485
[CARE_PROVIDER, INVESTIGATOR, OUTCOMES_ASSESSOR, PARTICIPANT]    13694
[INVESTIGATOR, PARTICIPANT]                                       8711
[CARE_PROVIDER, INVESTIGATOR, PARTICIPANT]                        3868
[INVESTIGATOR, OUTCOMES_ASSESSOR, PARTICIPANT]                    2700
[PARTICIPANT]                                                     2390
[OUTCOMES_ASSESSOR]                                               2048
[OUTCOMES_ASSESSOR, PARTICIPANT]                                  1072
[CARE_PROVIDER, PARTICIPANT]                                       762
[INVESTIGATOR]                                                     680
[CARE_PROVIDER, OUTCOMES_ASSESSOR, PARTICIPANT]                    532
[INVESTIGATOR, OUTCOMES_ASSESSOR]                                  345
[CARE_PROVIDER]                                                     98
[CARE_PROVIDER, INVESTIGATOR, OUTCOMES_ASSESSOR]         

In [54]:
df_temp = df.copy()
df_temp["Masking_Detail_List"] = df_temp["Masking_Detail_List"].apply(tuple)

counts = (df_temp[["Masking_Detail_List", "Masking_List"]]
    .value_counts().reset_index(name="count"))
counts

,Masking_Detail_List,Masking_List,count
0,"(MASK_DET_NONE,)",MASK_NONE,41485
1,"(PARTICIPANT, INVESTIGATOR, CARE_PROVIDER, OUT...",QUADRUPLE,13694
2,"(INVESTIGATOR, PARTICIPANT)",DOUBLE,8711
3,"(INVESTIGATOR, CARE_PROVIDER, PARTICIPANT)",TRIPLE,3868
4,"(PARTICIPANT, INVESTIGATOR, OUTCOMES_ASSESSOR)",TRIPLE,2700
5,"(PARTICIPANT,)",SINGLE,2390
6,"(OUTCOMES_ASSESSOR,)",SINGLE,2048
7,"(PARTICIPANT, OUTCOMES_ASSESSOR)",DOUBLE,1072
8,"(PARTICIPANT, CARE_PROVIDER)",DOUBLE,762
9,"(INVESTIGATOR,)",SINGLE,680


### Primary Purpose_List

In [55]:
df = df.merge(my_designs[["nct_id", "primary_purpose"]], on = "nct_id", how = "left")
df = df.rename(columns={"primary_purpose" : "Primary_Purpose_List"})
display(df["Primary_Purpose_List"].value_counts(dropna=False))


Primary_Purpose_List
TREATMENT                   59304
PREVENTION                   7064
BASIC_SCIENCE                4216
OTHER                        3403
DIAGNOSTIC                   1533
SUPPORTIVE_CARE              1497
None                          980
HEALTH_SERVICES_RESEARCH      415
SCREENING                     169
DEVICE_FEASIBILITY             30
Name: count, dtype: int64

#### NaN+ -> Other
- Device had too little rows --> Possible sparse data problem 

In [56]:
df["Primary_Purpose_List"] = df["Primary_Purpose_List"].replace({np.nan : "PRIM_PURP_OTHER", "OTHER" : "PRIM_PURP_OTHER"})
display(df["Primary_Purpose_List"].value_counts(dropna=False))


Primary_Purpose_List
TREATMENT                   59304
PREVENTION                   7064
PRIM_PURP_OTHER              4383
BASIC_SCIENCE                4216
DIAGNOSTIC                   1533
SUPPORTIVE_CARE              1497
HEALTH_SERVICES_RESEARCH      415
SCREENING                     169
DEVICE_FEASIBILITY             30
Name: count, dtype: int64

#### Merge

In [57]:
purp_map = { 
    'DEVICE_FEASIBILITY' : 'PRIM_PURP_OTHER',
    'INTERV_OTHER' : 'PRIM_PURP_OTHER',
    'SCREENING' : 'PRIM_PURP_OTHER',
    'HEALTH_SERVICES_RESEARCH' : 'PRIM_PURP_OTHER'
}
df['Primary_Purpose_List'] = df['Primary_Purpose_List'].replace(purp_map)
df['Primary_Purpose_List'].value_counts()

Primary_Purpose_List
TREATMENT          59304
PREVENTION          7064
PRIM_PURP_OTHER     4997
BASIC_SCIENCE       4216
DIAGNOSTIC          1533
SUPPORTIVE_CARE     1497
Name: count, dtype: int64

## Arm_Counts

In [58]:
df = df.merge(my_studies[["nct_id", "number_of_arms"]], on = "nct_id", how = "left")
df = df.rename(columns = {"number_of_arms" : "Arm_Counts"})
display(df['Arm_Counts'].value_counts(dropna = False))


Arm_Counts
2.0     37678
1.0     19189
3.0      9798
4.0      5915
5.0      1886
6.0      1668
7.0       538
8.0       519
NaN       307
9.0       287
10.0      229
12.0      140
11.0      113
13.0       73
14.0       61
15.0       49
16.0       41
18.0       30
17.0       19
19.0       12
20.0       11
22.0        9
21.0        7
24.0        5
23.0        5
25.0        4
27.0        4
32.0        3
26.0        2
30.0        2
37.0        1
29.0        1
31.0        1
44.0        1
28.0        1
43.0        1
34.0        1
Name: count, dtype: int64

#### NaN -> Drop

In [59]:
df = df.dropna(subset = ["Arm_Counts"])
df['Arm_Counts'] = df['Arm_Counts'].astype(int)
display(df['Arm_Counts'].value_counts(dropna = False))

Arm_Counts
2     37678
1     19189
3      9798
4      5915
5      1886
6      1668
7       538
8       519
9       287
10      229
12      140
11      113
13       73
14       61
15       49
16       41
18       30
17       19
19       12
20       11
22        9
21        7
24        5
23        5
25        4
27        4
32        3
26        2
30        2
37        1
29        1
31        1
44        1
28        1
43        1
34        1
Name: count, dtype: int64

### Arm_Counts_Log

In [60]:
df['Arm_Counts_Log'] = np.log1p(df["Arm_Counts"]) 
df['Arm_Counts_Log'].describe()

count    78304.000000
mean         1.143119
std          0.374654
min          0.693147
25%          1.098612
50%          1.098612
75%          1.386294
max          3.806662
Name: Arm_Counts_Log, dtype: float64

## Locations

ClinicalTrials.gov Documenation:
•	Location
A place where a research site for a clinical study can be found. Location information can be searched using a facility name, a city, state, zip code, or country. A location where a study is being conducted may also include contact information.

ClinicalTrials.gov Documenation:
•	Location countries
Countries in which research facilities for a study are located. A country is listed only once, even if there is more than one facility in the country. The list includes all countries as of the last update submitted date; any country for which all facilities were removed from the study record are listed under removed location countries.


### Countries

In [61]:
df = df.merge(my_locations[["nct_id", "facilities_country"]], on = "nct_id", how = "left")
df = df.rename(columns = {"facilities_country": "Countries_List"})
df['Countries_List'].value_counts(dropna = False)

Countries_List
[United States]                                                                                                                         27463
[China]                                                                                                                                  5042
[]                                                                                                                                       4419
[South Korea]                                                                                                                            2443
[United Kingdom]                                                                                                                         2246
                                                                                                                                        ...  
[Belgium, Czechia, Israel, Italy, Netherlands, Romania, Russia, Slovakia]                                                            

#### NaN ++ -> Other

In [62]:
df['Countries_List'] = df['Countries_List'].apply(lambda x: ['Countr_Other'] if len(x) == 0 else x)

### Country_Counts


In [63]:
df = df.merge(my_locations[["nct_id", "Country_Counts"]], on = "nct_id", how = "left")
display(df["Country_Counts"].value_counts(dropna=False))
display(df.shape) 

Country_Counts
1     61203
0      4419
2      3430
3      1495
4      1158
5       932
6       796
7       679
8       582
9       448
10      427
11      349
12      297
13      251
14      230
15      200
16      182
18      150
17      137
19      132
20       97
21       86
22       83
23       68
24       61
26       53
25       49
28       38
27       36
29       35
31       33
30       24
32       21
33       19
34       15
37       13
35       11
42        9
41        8
36        8
38        7
49        5
44        5
40        5
43        5
48        3
45        3
46        2
39        2
59        1
57        1
51        1
Name: count, dtype: int64

(78304, 64)

### Country_Counts_Log

In [64]:
df['Country_Counts_Log'] = np.log1p(df['Country_Counts'])

### Continents_List

In [65]:
Countries = pd.DataFrame(geonamescache.GeonamesCache().get_countries()).transpose().reset_index()
Countries = Countries[["name", "continentcode"]]
Countries = Countries.replace({"NA": "North America", "OC": "Oceania", "EU": "Europe", "AS": "Asia", "AF": "Africa", "SA": "South America"})
geo_dict = dict(zip( Countries['name'], Countries['continentcode']))

#### NaN++ --> Fillna + Other

In [66]:
df['Continents_List'] = df['Countries_List'].apply(
    lambda x: [geo_dict.get(item, 'None') for item in x] if isinstance(x, list) else ['Cont_Other'])

df['Continents_List'] = df['Continents_List'].apply(lambda x: [i for i in x if i != 'None']) # matched some '' from geodict
df['Continents_List'] = df['Continents_List'].apply(lambda x: ['Cont_Other'] if len(x) == 0 else x)
df['Continents_List'] = df['Continents_List'].apply(lambda x: sorted(set(x)))

display(df['Continents_List'].value_counts(dropna = False))


Continents_List
[North America]                                                  31103
[Europe]                                                         15345
[Asia]                                                           13188
[Cont_Other]                                                      4852
[Europe, North America]                                           2620
[Africa]                                                          2461
[South America]                                                   1158
[Asia, Europe, North America]                                     1153
[Oceania]                                                         1009
[Asia, Europe, North America, Oceania]                             764
[Europe, North America, Oceania]                                   586
[Asia, Europe, North America, Oceania, South America]              513
[Asia, Europe, North America, South America]                       390
[Asia, Europe]                                               

#### Merge
- Based on Sparse file

In [67]:
cont_map = { 
    'Oceania' : 'Cont_Other',
    'Africa' : 'Cont_Other',
    'South America' : 'Cont_Other'   
}
display(Counter([elem for row_list in df["Continents_List"] for elem in row_list]))
df['Continents_List'] = df['Continents_List'].apply(lambda row: [cont_map.get(val, val) for val in row])
display(Counter([elem for row_list in df["Continents_List"] for elem in row_list]))


Counter({'North America': 39365,
         'Europe': 23374,
         'Asia': 17937,
         'Cont_Other': 4852,
         'Oceania': 4076,
         'Africa': 3699,
         'South America': 3273})

Counter({'North America': 39365,
         'Europe': 23374,
         'Asia': 17937,
         'Cont_Other': 15900})

### Continent_Counts
####  NaNs+ --> 0

In [68]:
df["Continent_Counts"] = df["Continents_List"].apply(lambda x: len(x) if isinstance(x, list) and x != ['Cont_Other'] else 0)
display(df["Continent_Counts"].value_counts(dropna = False))
display(df.shape)

Continent_Counts
1    59636
0     9480
2     4174
3     2362
4     1521
5      844
6      287
Name: count, dtype: int64

(78304, 67)

### Continent_Count_Log

In [69]:
df['Continent_Counts_Log'] = np.log1p(df['Continent_Counts'])

### City_Counts


In [70]:
df = df.merge(my_locations[[ "nct_id", "City_Counts"]], on = "nct_id", how = "left") 
# Cities already sorted(set(x)) in df_locations --> correct counts
display(df["City_Counts"].value_counts(dropna=False))
display(df.shape) 

City_Counts
1      44411
0       4419
2       4384
3       2587
4       1950
       ...  
243        1
441        1
338        1
298        1
272        1
Name: count, Length: 402, dtype: int64

(78304, 69)

### City_Counts_Log

In [71]:
df['City_Counts_Log'] = np.log1p(df['City_Counts'])

## Enrollment


### Enrollment_Counts

In [72]:
df = df.rename(columns = {"Enrollment": "Enrollment_Counts"})
display(df["Enrollment_Counts"].value_counts(dropna=False)) 

Enrollment_Counts
0.0       4374
20.0      1714
24.0      1704
30.0      1698
40.0      1540
          ... 
3250.0       1
7075.0       1
1341.0       1
1061.0       1
3231.0       1
Name: count, Length: 2269, dtype: int64

#### NaNs --> Fillna db + Drop
- No values could be filled from my_terminations table. 

In [ ]:
display(df['Enrollment_Counts'].isnull().sum())
# display(df.loc[df['Enrollment_Counts'].isnull()][['Enrollment_Counts', 'Study_Status_Bin']].value_counts()) # was also checked if withdrawn 
df = df.merge(my_studies[['nct_id', 'enrollment']], on = 'nct_id', how = 'left')
df['Enrollment_Counts'] = df['Enrollment_Counts'].fillna(df['enrollment']) # Nothing filled from db

display(df['Enrollment_Counts'].isnull().sum())

df = df.dropna(subset = ["Enrollment_Counts"])
df['Enrollment_Counts'] = df['Enrollment_Counts'].astype(int)

df['Enrollment_Counts'].isnull().sum()

3

Series([], Name: count, dtype: int64)

3

0

### Enrollment_Log

In [75]:
df['Enrollment_Counts_Log'] = np.log1p(df['Enrollment_Counts'])
df['Enrollment_Counts_Log'].describe()  

count    78301.000000
mean         3.811241
std          1.632357
min          0.000000
25%          2.995732
50%          3.850148
75%          4.795791
max         13.669900
Name: Enrollment_Counts_Log, dtype: float64

# Drop Columns

In [76]:
df = df.drop(columns = [
       "nct_id",  # Unique identifier not feature, no longer needed for analysis
       'Brief Summary', 
       'Study Title', 'Study URL', 'Acronym', 'Other IDs',  # Unique identifier not feature
       'official_title', 
       'Conditions', 'Interventions', # text data used in other format
       'Primary Outcome Measures', 'Secondary Outcome Measures', 'Other Outcome Measures', 'Study Results', # Results not pre-designed features
       'Sponsor', 'Collaborators', # used in other format 
       'Study Type', # All interventional
       'Study Design',  # used in other format
       'Start Date', 'Primary Completion Date', 'Completion Date', 'First Posted', # used in other format
       'Results First Posted', 'Last Update Posted', 
       'Locations', 'Countries_List', # used in other format
       'Study Documents', 
       'Injection', 'Oral', 'Topical', 'Surgical', 
       'enrollment',
       ], axis = 1)

df.columns

Index(['Study_Status_Bin', 'Sex_List', 'Age_List', 'Phases_List',
       'Enrollment_Counts', 'Funder_Industry_Bin', 'Start_Date_Year_Counts',
       'Completion_Date_Year_Counts', 'Completion_Gap_Counts',
       'Completion_Gap_Log', 'Intervention_Type_List',
       'Intervention_Type_Counts', 'Intervention_Route_List',
       'Intervention_Route_Counts', 'Placebo_Bin', 'Standard_Care_Bin',
       'Healthy_Bin', 'Covid_19_Bin', 'Conditions_Detail_List', 'Adverse_List',
       'Adverse_Counts', 'Adverse_Counts_Log', 'Adverse_Bin',
       'Adverse_System_List', 'Adverse_System_Counts',
       'Adverse_System_Counts_Log', 'Allocation_List',
       'Intervention_Model_List', 'Masking_List', 'Masking_Detail_List',
       'Primary_Purpose_List', 'Arm_Counts', 'Arm_Counts_Log',
       'Country_Counts', 'Country_Counts_Log', 'Continents_List',
       'Continent_Counts', 'Continent_Counts_Log', 'City_Counts',
       'City_Counts_Log', 'Enrollment_Counts_Log'],
      dtype='object')

# Remaining NaNs
- SOS Only after dropping columns

In [77]:
display(df.isnull().sum())
df = df.dropna()

Study_Status_Bin               0
Sex_List                       0
Age_List                       0
Phases_List                    0
Enrollment_Counts              0
Funder_Industry_Bin            0
Start_Date_Year_Counts         0
Completion_Date_Year_Counts    0
Completion_Gap_Counts          0
Completion_Gap_Log             0
Intervention_Type_List         0
Intervention_Type_Counts       0
Intervention_Route_List        0
Intervention_Route_Counts      0
Placebo_Bin                    0
Standard_Care_Bin              0
Healthy_Bin                    0
Covid_19_Bin                   0
Conditions_Detail_List         0
Adverse_List                   0
Adverse_Counts                 0
Adverse_Counts_Log             0
Adverse_Bin                    0
Adverse_System_List            0
Adverse_System_Counts          0
Adverse_System_Counts_Log      0
Allocation_List                0
Intervention_Model_List        0
Masking_List                   0
Masking_Detail_List            0
Primary_Pu

# Dfs Split

In [78]:
df_dum = df.copy()
mlb = MultiLabelBinarizer() # encode list elements
dummies = pd.DataFrame(mlb.fit_transform(df['Phases_List']), columns = mlb.classes_, index = df_dum.index)
df_dum = pd.concat([df_dum, dummies] , axis = 1)
df_dum.head(2)

,Study_Status_Bin,Sex_List,Age_List,Phases_List,Enrollment_Counts,Funder_Industry_Bin,Start_Date_Year_Counts,Completion_Date_Year_Counts,Completion_Gap_Counts,Completion_Gap_Log,Intervention_Type_List,Intervention_Type_Counts,Intervention_Route_List,Intervention_Route_Counts,Placebo_Bin,Standard_Care_Bin,Healthy_Bin,...,Masking_List,Masking_Detail_List,Primary_Purpose_List,Arm_Counts,Arm_Counts_Log,Country_Counts,Country_Counts_Log,Continents_List,Continent_Counts,Continent_Counts_Log,City_Counts,City_Counts_Log,Enrollment_Counts_Log,PHASE1,PHASE2,PHASE3,PHASE4
0,COMPLETED,MALE,[ADULT],[PHASE1],24,INDUSTRY,2014,2014,2,1.098612,[DRUG],1,[Inter_Route_Other],1,No,No,Yes,...,MASK_NONE,[MASK_DET_NONE],TREATMENT,2,1.098612,1,0.693147,[Asia],1,0.693147,1,0.693147,3.218876,1,0,0,0
1,COMPLETED,MALE,[ADULT],[PHASE1],39,FUNDER_OTHER,2013,2013,9,2.302585,[DRUG],1,[Inter_Route_Other],1,Yes,No,Yes,...,MASK_NONE,[MASK_DET_NONE],DIAGNOSTIC,3,1.386294,1,0.693147,[Asia],1,0.693147,1,0.693147,3.688879,1,0,0,0


In [79]:
df1 = df_dum[df_dum["PHASE1"] == 1].reset_index(drop = True).drop(columns = ['PHASE1','PHASE2', 'PHASE3', 'PHASE4'])
df2 = df_dum[df_dum["PHASE2"] == 1].reset_index(drop = True).drop(columns = ['PHASE1','PHASE2', 'PHASE3', 'PHASE4'])
df3 = df_dum[df_dum["PHASE3"] == 1].reset_index(drop = True).drop(columns = ['PHASE1','PHASE2', 'PHASE3', 'PHASE4'])
df4 = df_dum[df_dum["PHASE4"] == 1].reset_index(drop = True).drop(columns = ['PHASE1','PHASE2', 'PHASE3', 'PHASE4'])

df1 = df1.drop(columns = ['Phases_List'], axis = 1)
df2 = df2.drop(columns = ['Phases_List'], axis = 1)
df3 = df3.drop(columns = ['Phases_List'], axis = 1)
df4 = df4.drop(columns = ['Phases_List'], axis = 1)

display(df1.shape)
display(df2.shape)
display(df3.shape)
display(df4.shape)


(27324, 40)

(29260, 40)

(16503, 40)

(13246, 40)

# Categ_Columns

## Functions

In [80]:
# dfis
dfis = [df1, df2, df3, df4]

In [81]:
# def_describe
def def_describe(col):
    describes = pd.concat([
        df1[col].describe(include = 'all'),
        df2[col].describe(include = 'all'),
        df3[col].describe(include = 'all'),
        df4[col].describe(include = 'all'),
    ], axis=1)

    describes.columns = ['df1', 'df2', 'df3', 'df4']
    describes.index.name = col
    return describes


In [82]:
# fun_bins
def fun_bins(dfis, col, bin2_val, use_quantile=False):
    bins = []
    labels = []

    for dfi in dfis:
        bin1 = dfi[col].min()
        bin2 = dfi[col].quantile(bin2_val) if use_quantile else bin2_val
        bin3 = dfi[col].max()

        bins.append([bin1, bin2, bin3])
        labels.append([f"{int(bin1)}–{int(bin2)}", f"{int(bin2)+1}–{int(bin3)}"])

    return bins, labels

In [83]:
def fun_bins2(dfis, col, bin1_val, bin2_val):
   
    bins = []
    labels = []

    for dfi in dfis:
        bin1 = bin1_val  # when min() = median() e.g., 1 --> bin1 = 0, bin2 = 1
        bin2 = bin2_val
        bin3 = dfi[col].max()

        bins.append([bin1, bin2, bin3])
        labels.append([f"{int(bin2_val)}", f"{int(bin2_val)+1}–{int(bin3)}"]) # there is usually no 0 in bvalues

    return bins, labels

## Enrollment_Categ

In [84]:
Describe_Enrollment = def_describe('Enrollment_Counts')
display(Describe_Enrollment)

col = 'Enrollment_Counts'
col_categ = 'Enrollment_Categ'
bins, labels = fun_bins(dfis, col, bin2_val = 0.5, use_quantile = True)

df1[col_categ] = pd.cut(df1[col], bins = bins[0], labels = labels[0], ordered = True, include_lowest = True, right = True)
df2[col_categ] = pd.cut(df2[col], bins = bins[1], labels = labels[1], ordered = True, include_lowest = True, right = True)
df3[col_categ] = pd.cut(df3[col], bins = bins[2], labels = labels[2], ordered = True, include_lowest = True, right = True)
df4[col_categ] = pd.cut(df4[col], bins = bins[3], labels = labels[3], ordered = True, include_lowest = True, right = True)

df1["Enrollment_Categ"].value_counts(dropna = False)


,df1,df2,df3,df4
Enrollment_Counts,,,,
count,27324.000000,29260.000000,16503.000000,13246.000000
mean,46.986495,110.912714,542.926559,470.627737
std,107.271282,1085.208354,3266.428533,9615.979740
min,0.000000,0.000000,0.000000,0.000000
25%,14.000000,15.000000,51.000000,25.000000
50%,28.000000,41.000000,158.000000,60.500000
75%,51.000000,100.000000,417.000000,140.000000
max,8231.000000,144539.000000,204438.000000,864493.000000


Enrollment_Categ
0–28       13978
29–8231    13346
Name: count, dtype: int64

## Adverse_Categ
- Most distributions are around 0, thus if categorised it is equivalent to Adverse Bin. 


In [85]:
Describe_Adverse = def_describe('Adverse_Counts')
Describe_Adverse

# Distribution of Adverse_Counts is up to 75% at 0. Only Adverse_Bin is going to be used.

,df1,df2,df3,df4
Adverse_Counts,,,,
count,27324.000000,29260.000000,16503.000000,13246.000000
mean,9.932806,34.175769,168.660304,28.577004
std,62.074198,130.253461,770.036362,652.944694
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.000000,22.000000,93.000000,0.000000
max,3385.000000,5862.000000,47910.000000,70245.000000


## Adverse_System_Categ
Is about counts of Adverse_System = equal to adverse_categ as they are both combined to count format.
Does not have meaning (<=1 , >1) as its similar to Adverse_Bin --> has does not have --> wont be  used



In [86]:
Describe_Adverse_System = def_describe('Adverse_System_Counts')
display(Describe_Adverse_System)

col = 'Adverse_System_Counts'
col_categ = 'Adverse_System_Categ'
bins, labels = fun_bins(dfis, col, bin2_val = 1, use_quantile = False)  

df1[col_categ] = pd.cut(df1[col], bins = bins[0], labels = labels[0], ordered = True, include_lowest = True, right = True)
df2[col_categ] = pd.cut(df2[col], bins = bins[1], labels = labels[1], ordered = True, include_lowest = True, right = True)
df3[col_categ] = pd.cut(df3[col], bins = bins[2], labels = labels[2], ordered = True, include_lowest = True, right = True)
df4[col_categ] = pd.cut(df4[col], bins = bins[3], labels = labels[3], ordered = True, include_lowest = True, right = True)

display(df1['Adverse_System_Categ'].value_counts(dropna = False))

,df1,df2,df3,df4
Adverse_System_Counts,,,,
count,27324.000000,29260.000000,16503.000000,13246.000000
mean,2.176585,4.615106,5.187178,1.498717
std,5.137081,6.768148,7.125980,3.748889
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.000000,9.000000,10.000000,1.000000
max,22.000000,22.000000,22.000000,22.000000


Adverse_System_Categ
0–1     22017
2–22     5307
Name: count, dtype: int64

## Arm_Categ
- See thsesis_Vizual --> Many data points in specific values. 1-2 many data points and less in >=3. + Skewed --> Visible change after 2 (=median in all dfs) --> Better categorical.


In [87]:
Describe_Arm = def_describe('Arm_Counts')
display(Describe_Arm)

col = 'Arm_Counts'
col_categ = 'Arm_Categ'
bins, labels = fun_bins(dfis, col, bin2_val = 2, use_quantile = False)  
# 0.5 bind do not increase monotonically. Use fixed bin2_val = 2

df1[col_categ] = pd.cut(df1[col], bins = bins[0], labels = labels[0], ordered = True, include_lowest = True, right = True)
df2[col_categ] = pd.cut(df2[col], bins = bins[1], labels = labels[1], ordered = True, include_lowest = True, right = True)
df3[col_categ] = pd.cut(df3[col], bins = bins[2], labels = labels[2], ordered = True, include_lowest = True, right = True)
df4[col_categ] = pd.cut(df4[col], bins = bins[3], labels = labels[3], ordered = True, include_lowest = True, right = True)

display(df1['Arm_Categ'].value_counts(dropna = False))

,df1,df2,df3,df4
Arm_Counts,,,,
count,27324.000000,29260.000000,16503.000000,13246.000000
mean,2.755490,2.335919,2.263891,2.089989
std,2.396482,1.752299,1.162366,0.988447
min,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,2.000000,2.000000
50%,2.000000,2.000000,2.000000,2.000000
75%,3.000000,3.000000,2.000000,2.000000
max,44.000000,43.000000,32.000000,32.000000


Arm_Categ
1–2     17491
3–44     9833
Name: count, dtype: int64

## Intervention_Type_Categ

In [88]:
Describe_Intervention_Type = def_describe('Intervention_Type_Counts')
display(Describe_Intervention_Type)

col = 'Intervention_Type_Counts'
col_categ = 'Intervention_Type_Categ'

bins, labels = fun_bins2(dfis, col, bin1_val = 0.5 , bin2_val = 1)

df1[col_categ] = pd.cut(df1[col], bins = bins[0], labels = labels[0], ordered = True, include_lowest = True, right = True)
df2[col_categ] = pd.cut(df2[col], bins = bins[1], labels = labels[1], ordered = True, include_lowest = True, right = True)
df3[col_categ] = pd.cut(df3[col], bins = bins[2], labels = labels[2], ordered = True, include_lowest = True, right = True)
df4[col_categ] = pd.cut(df4[col], bins = bins[3], labels = labels[3], ordered = True, include_lowest = True, right = True)

display(df1['Intervention_Type_Categ'].value_counts())

,df1,df2,df3,df4
Intervention_Type_Counts,,,,
count,27324.000000,29260.000000,16503.000000,13246.000000
mean,1.075867,1.106972,1.077380,1.089461
std,0.290748,0.345031,0.282417,0.304365
min,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000
max,4.000000,5.000000,4.000000,4.000000


Intervention_Type_Categ
1      25437
2–4     1887
Name: count, dtype: int64

## Intervention_Route_Categ

In [89]:
Describe_Intervention_Route = def_describe('Intervention_Route_Counts')
display(Describe_Intervention_Route)

col = 'Intervention_Route_Counts'
col_categ = 'Intervention_Route_Categ'

bins, labels = fun_bins2(dfis, col, bin1_val = 0.5, bin2_val = 1)

df1[col_categ] = pd.cut(df1[col], bins = bins[0], labels = labels[0], ordered = True, include_lowest = True, right = True)
df2[col_categ] = pd.cut(df2[col], bins = bins[1], labels = labels[1], ordered = True, include_lowest = True, right = True)
df3[col_categ] = pd.cut(df3[col], bins = bins[2], labels = labels[2], ordered = True, include_lowest = True, right = True)
df4[col_categ] = pd.cut(df4[col], bins = bins[3], labels = labels[3], ordered = True, include_lowest = True, right = True)

display(df1['Intervention_Route_Categ'].value_counts())

,df1,df2,df3,df4
Intervention_Route_Counts,,,,
count,27324.000000,29260.000000,16503.000000,13246.000000
mean,1.172998,1.218524,1.185906,1.286426
std,0.425087,0.471360,0.434655,0.516347
min,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,2.000000
max,4.000000,4.000000,4.000000,4.000000


Intervention_Route_Categ
1      23106
2–4     4218
Name: count, dtype: int64

## City_Categ

In [90]:
Describe_City = def_describe('City_Counts')
display(Describe_City)

col = 'City_Counts'
col_categ = 'City_Categ'

bins, labels = fun_bins(dfis, col, bin2_val = 1, use_quantile = False)
# Almost all distribution at 1, so use fixed bin2_val = 1

df1[col_categ] = pd.cut(df1[col], bins = bins[0], labels = labels[0], ordered = True, include_lowest = True, right = True)
df2[col_categ] = pd.cut(df2[col], bins = bins[1], labels = labels[1], ordered = True, include_lowest = True, right = True)
df3[col_categ] = pd.cut(df3[col], bins = bins[2], labels = labels[2], ordered = True, include_lowest = True, right = True)
df4[col_categ] = pd.cut(df4[col], bins = bins[3], labels = labels[3], ordered = True, include_lowest = True, right = True)

display(df1['City_Categ'].value_counts(dropna = False))

,df1,df2,df3,df4
City_Counts,,,,
count,27324.000000,29260.000000,16503.000000,13246.000000
mean,2.865905,9.018763,26.384112,4.325985
std,6.140913,22.804192,57.239258,17.885615
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,2.000000,1.000000
75%,2.000000,8.000000,28.000000,1.000000
max,282.000000,667.000000,1064.000000,607.000000


City_Categ
0–1      19496
2–282     7828
Name: count, dtype: int64

## Country_Categ

In [91]:
Describe_Country = def_describe('Country_Counts')
display(Describe_Country)

col = 'Country_Counts'
col_categ = 'Country_Categ'

bins, labels = fun_bins(dfis, col, bin2_val = 1, use_quantile = False)
# Almost all distribution is at 1 country. Use fixed bin2_val = 1

df1[col_categ] = pd.cut(df1[col], bins = bins[0], labels = labels[0], ordered = True, include_lowest = True, right = True)
df2[col_categ] = pd.cut(df2[col], bins = bins[1], labels = labels[1], ordered = True, include_lowest = True, right = True)
df3[col_categ] = pd.cut(df3[col], bins = bins[2], labels = labels[2], ordered = True, include_lowest = True, right = True)
df4[col_categ] = pd.cut(df4[col], bins = bins[3], labels = labels[3], ordered = True, include_lowest = True, right = True)

display(df1['Country_Categ'].value_counts(dropna = False))

,df1,df2,df3,df4
Country_Counts,,,,
count,27324.000000,29260.000000,16503.000000,13246.000000
mean,1.236276,1.842208,3.732109,1.234033
std,1.208108,2.781704,6.348114,1.948536
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,3.000000,1.000000
max,33.000000,46.000000,59.000000,42.000000


Country_Categ
0–1     24569
2–33     2755
Name: count, dtype: int64

## Continent_Categ

In [92]:
Describe_Continent = def_describe('Continent_Counts')
display(Describe_Continent)

col = 'Continent_Counts'
col_categ = 'Continent_Categ'

bins, labels = fun_bins(dfis, col, bin2_val = 1, use_quantile = False)
# Almost all distribution is at 1 continent. Use fixed bin2_val = 1

df1[col_categ] = pd.cut(df1[col], bins = bins[0], labels = labels[0], ordered = True, include_lowest = True, right = True)
df2[col_categ] = pd.cut(df2[col], bins = bins[1], labels = labels[1], ordered = True, include_lowest = True, right = True)
df3[col_categ] = pd.cut(df3[col], bins = bins[2], labels = labels[2], ordered = True, include_lowest = True, right = True)
df4[col_categ] = pd.cut(df4[col], bins = bins[3], labels = labels[3], ordered = True, include_lowest = True, right = True)

display(df1['Continent_Categ'].value_counts(dropna = False))

,df1,df2,df3,df4
Continent_Counts,,,,
count,27324.000000,29260.000000,16503.000000,13246.000000
mean,1.004648,1.123753,1.432649,0.870074
std,0.525610,0.785887,1.304433,0.568016
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000
max,6.000000,6.000000,6.000000,6.000000


Continent_Categ
0–1    25388
2–6     1936
Name: count, dtype: int64

## Completion_Gap_Categ


In [93]:
Describe_Completion_Gap = def_describe('Completion_Gap_Counts')
display(Describe_Completion_Gap)

col = 'Completion_Gap_Counts'
col_categ = 'Completion_Gap_Categ'

bins, labels = fun_bins(dfis, col, bin2_val = 0.5, use_quantile = True)

df1[col_categ] = pd.cut(df1[col], bins = bins[0], labels = labels[0], ordered = True, include_lowest = True, right = True)
df2[col_categ] = pd.cut(df2[col], bins = bins[1], labels = labels[1], ordered = True, include_lowest = True, right = True)
df3[col_categ] = pd.cut(df3[col], bins = bins[2], labels = labels[2], ordered = True, include_lowest = True, right = True)
df4[col_categ] = pd.cut(df4[col], bins = bins[3], labels = labels[3], ordered = True, include_lowest = True, right = True)

display(df1['Completion_Gap_Categ'].value_counts())

,df1,df2,df3,df4
Completion_Gap_Counts,,,,
count,27324.0,29260.0,16503.0,13246.0
mean,22.43804,31.8608,29.700782,25.543334
std,23.421838,24.308621,22.874919,20.092806
min,0.0,0.0,0.0,0.0
25%,5.0,13.0,13.0,11.0
50%,14.0,26.0,24.0,21.0
75%,33.0,44.0,40.0,35.0
max,160.0,163.0,162.0,149.0


Completion_Gap_Categ
0–14      13940
15–160    13384
Name: count, dtype: int64

## Start_Date_Year_Categ

In [94]:
Describe_Start_Date_Year = def_describe('Start_Date_Year_Counts')
display(Describe_Start_Date_Year)

col = 'Start_Date_Year_Counts'
col_categ = 'Start_Date_Year_Categ'

bins = [2011, 2018, 2024] # Covid oriented dates
labels = ['2011-2018', '2019-2024']

df1[col_categ] = pd.cut(df1[col], bins = bins, labels = labels, ordered = True, include_lowest = True, right = True)
df2[col_categ] = pd.cut(df2[col], bins = bins, labels = labels, ordered = True, include_lowest = True, right = True)
df3[col_categ] = pd.cut(df3[col], bins = bins, labels = labels, ordered = True, include_lowest = True, right = True)
df4[col_categ] = pd.cut(df4[col], bins = bins, labels = labels, ordered = True, include_lowest = True, right = True)

display(df1['Start_Date_Year_Categ'].value_counts(dropna = False))

,df1,df2,df3,df4
Start_Date_Year_Counts,,,,
count,27324.0,29260.0,16503.0,13246.0
mean,2016.747365,2016.259638,2016.021087,2016.058735
std,3.669283,3.46797,3.495488,3.454502
min,2011.0,2011.0,2011.0,2011.0
25%,2014.0,2013.0,2013.0,2013.0
50%,2017.0,2016.0,2016.0,2016.0
75%,2020.0,2019.0,2019.0,2019.0
max,2024.0,2024.0,2024.0,2024.0


Start_Date_Year_Categ
2011-2018    17745
2019-2024     9579
Name: count, dtype: int64

## Completion_Date_Year_Categ

In [95]:
Describe_Completion_Date_Year = def_describe('Completion_Date_Year_Counts')
display(Describe_Completion_Date_Year)

col = 'Completion_Date_Year_Counts'
col_categ = 'Completion_Date_Year_Categ'

bins = [2011, 2018, 2024] # Covid oriented dates
labels = ['2011-2018', '2019-2024']

df1[col_categ] = pd.cut(df1[col], bins = bins, labels = labels, ordered = True, include_lowest = True, right = True)
df2[col_categ] = pd.cut(df2[col], bins = bins, labels = labels, ordered = True, include_lowest = True, right = True)
df3[col_categ] = pd.cut(df3[col], bins = bins, labels = labels, ordered = True, include_lowest = True, right = True)
df4[col_categ] = pd.cut(df4[col], bins = bins, labels = labels, ordered = True, include_lowest = True, right = True)

display(df1['Completion_Date_Year_Categ'].value_counts(dropna = False))


,df1,df2,df3,df4
Completion_Date_Year_Counts,,,,
count,27324.0,29260.0,16503.0,13246.0
mean,2018.589152,2018.880417,2018.463128,2018.139967
std,3.711437,3.49302,3.576029,3.469508
min,2011.0,2011.0,2011.0,2011.0
25%,2016.0,2016.0,2015.0,2015.0
50%,2019.0,2019.0,2019.0,2018.0
75%,2022.0,2022.0,2022.0,2021.0
max,2024.0,2024.0,2024.0,2024.0


Completion_Date_Year_Categ
2019-2024    14618
2011-2018    12706
Name: count, dtype: int64

## Describe
- Choose a good form for the features, as they are almost all skewed.
- For the features having the most of distribution i.e., median is around lower values, e.g., 0,1,2, a categorical format is going to be used. 
- In contrast, values with wider range of values, and median in higher values, performing a more wide distribution, will be analysed as logarithm. Logarythm scale is chosen in order to correct the skeweness. 

In [96]:
Describe_Enrollment = Describe_Enrollment.loc['50%'].rename('Enrollment')
Describe_Adverse = Describe_Adverse.loc['50%'].rename('Adverse')
Describe_Adverse_System = Describe_Adverse_System.loc['50%'].rename('Adverse_System')
Describe_Arm = Describe_Arm.loc['50%'].rename('Arm')
Describe_Intervention_Type = Describe_Intervention_Type.loc['50%'].rename('Intervention_Type')
Describe_Intervention_Route = Describe_Intervention_Route.loc['50%'].rename('Intervention_Route')
Describe_City = Describe_City.loc['50%'].rename('City')
Describe_Country = Describe_Country.loc['50%'].rename('Country')
Describe_Continent = Describe_Continent.loc['50%'].rename('Continent')
Describe_Completion_Gap = Describe_Completion_Gap.loc['50%'].rename('Completion_Gap') # year dates not used, as they wont be used fo analysis

In [97]:

medians = pd.concat([
    Describe_Enrollment,
    Describe_Adverse,
    Describe_Adverse_System,
    Describe_Arm,
    Describe_Intervention_Type,
    Describe_Intervention_Route,
    Describe_City,
    Describe_Country,
    Describe_Continent,
    Describe_Completion_Gap,
], axis=1)

medians = pd.DataFrame(medians)
mean_row = medians.mean(axis=0)
median_row = medians.median(axis=0)

mean_row.name = "Mean"
median_row.name = 'Median'
medians = pd.concat([medians, mean_row.to_frame().T])
medians = pd.concat([medians, median_row.to_frame().T])

medians

,Enrollment,Adverse,Adverse_System,Arm,Intervention_Type,Intervention_Route,City,Country,Continent,Completion_Gap
df1,28.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,14.0
df2,41.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,26.0
df3,158.0,0.0,0.0,2.0,1.0,1.0,2.0,1.0,1.0,24.0
df4,60.5,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,21.0
Mean,71.875,0.0,0.0,2.0,1.0,1.0,1.25,1.0,1.0,21.25
Median,50.75,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,22.5


# Save dfis

In [98]:
df.to_pickle(r".\df_data\df.pkl")  # pickle better to save dtypes e.g category, period etc. --> csv does not save dtypes
df1.to_pickle(r".\df_data\df1.pkl")
df2.to_pickle(r".\df_data\df2.pkl")
df3.to_pickle(r".\df_data\df3.pkl")
df4.to_pickle(r".\df_data\df4.pkl")
medians.to_pickle(r".\df_data\medians.pkl")

# Unused Columns

* Study Results: 
Was not used for analysis, as they are a result of a Study, not a pre-Designed Study characteristic.

* Primary Outcome Measures/Secondary Outcome Measures/Other Outcome Measures
- Was not used for analysis, as they are a result of a Study, not a pre-Designed Study characteristic.
- Primary Outcomes are usually pre-designed, as they are the main purpose of the trial. However, it is also associated with the condition/given drug for the condition field of thw trial. Thus, this study is not instrested in how investigators, express the condition/drug field and main outcome of this condition/drug field. 


* Study_Documents
ClinicalTrials.gov Documentation: https://clinicaltrials.gov/policy/protocol-definitions#a1-upload
Refers to the type of documents that the study sponsor or principal investigator may add to their study record. These include a study protocol, statistical analysis plan, and informed consent form.

- Was not be used for analysis because:
- Thousands of missing values. Almost 2/3 of total rows. 
- Seems data where just not uploaded to clinical trials.gov/db.
- As least Protocol is a nessecary predesigned document. Field seems optional to fill in registry.
- Study documents, especially protocol, are nessecary for a trial's design. The filling of this field was only made mandatory  after 2017 in Clinical Trials Registry. So most of the trials have missing values in this field. 

Documents: 
- ICF (Informed Consent Form)
- SAP (Statistical Analysis Plan)
- CSR (Clinical Study Report)  --> did not use because its table has less information for documents
- STUDY_PROTOCOL
- ANALYTIC_CODE --> did not use because its table has less information for documents

In [99]:
df.columns

Index(['Study_Status_Bin', 'Sex_List', 'Age_List', 'Phases_List',
       'Enrollment_Counts', 'Funder_Industry_Bin', 'Start_Date_Year_Counts',
       'Completion_Date_Year_Counts', 'Completion_Gap_Counts',
       'Completion_Gap_Log', 'Intervention_Type_List',
       'Intervention_Type_Counts', 'Intervention_Route_List',
       'Intervention_Route_Counts', 'Placebo_Bin', 'Standard_Care_Bin',
       'Healthy_Bin', 'Covid_19_Bin', 'Conditions_Detail_List', 'Adverse_List',
       'Adverse_Counts', 'Adverse_Counts_Log', 'Adverse_Bin',
       'Adverse_System_List', 'Adverse_System_Counts',
       'Adverse_System_Counts_Log', 'Allocation_List',
       'Intervention_Model_List', 'Masking_List', 'Masking_Detail_List',
       'Primary_Purpose_List', 'Arm_Counts', 'Arm_Counts_Log',
       'Country_Counts', 'Country_Counts_Log', 'Continents_List',
       'Continent_Counts', 'Continent_Counts_Log', 'City_Counts',
       'City_Counts_Log', 'Enrollment_Counts_Log'],
      dtype='object')